# A benchmark model of dynamic portfolio choice with transaction costs

In [6]:
import numpy as np

import torch
import torch.autograd as autograd

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

import cyipopt
from cyipopt import minimize_ipopt

from scipy.spatial import ConvexHull
from scipy.optimize import minimize
from numpy.polynomial.hermite import hermgauss

import matplotlib.pyplot as plt

np.random.seed(2001)

# Parameters
T = 10  # Time horizon
D = 2  # Number of risky assets
r = 0.03  # Risk-free return in pct.
# Rf = r
Rf = np.exp(r)  # Risk-free return
tau = 0.0025  # Transaction cost rate
beta = 0.975  # Discount factor
gamma = 3.0 # Risk aversion coefficient

# Risky assets - deterministic
mu = np.array([0.07, 0.07])
Sigma = np.array([[0.2, 0], [0, 0.2]])
Lambda = np.diag(np.sqrt(np.diag(Sigma)))


# Define the GPR model with ARD
class GPRegressionModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1]))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
def train_gp_model(train_x, train_y):
    likelihood = GaussianLikelihood(noise_constraint=gpytorch.constraints.GreaterThan(1e-6))  # Reduced noise variance
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 200
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward(retain_graph=True)
        optimizer.step()

    return model, likelihood

def utility(var, gamma):
    if gamma == 1:
        return torch.log(var)  # Log utility for risk aversion coefficient gamma = 1
    else:
        return (var**(1.0 - gamma)) / (1 - gamma)  # Power utility for other gamma values
    
def safe_utility(var, gamma):
    # Removed unnecessary re-wrapping of the tensor if it already has requires_grad=True
    var = torch.clamp(var, min=1e-10)  # Prevent log(0) or negative values
    return utility(var, gamma)

def normalized_bond_holdings(xt, delta_plus, delta_minus, tau):
    delta = delta_plus - delta_minus
    transaction_costs = tau * torch.sum(torch.abs(delta))
    # Compute the total risky asset holdings
    risky_assets = torch.sum(xt + delta)
    # Ensure bond holdings are valid and adjust accordingly
    bt = 1.0 - (risky_assets - transaction_costs)
    # If the risky assets and transaction costs exceed 1, make bond holdings zero (no borrowing)
    bt = torch.clamp(bt, min=0, max=1.0)
    return bt
def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    delta = delta_plus - delta_minus

    # equation 7
    pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
    pi_t1 = torch.clamp(pi_t1, min=1e-10)  # Avoid division by zero or negative wealth    
    
    # Equation 9
    xt1 = (xt + delta_plus - delta_minus) * Rt / pi_t1
    return pi_t1, xt1

def initialize_value_function(T, tau, gamma):
    V = [[None,None] for _ in range(T + 1)]

    def V_terminal(xT):
        return safe_utility(1 - tau * torch.sum(torch.abs(xT)), gamma)

    # Set both vt_next_in and vt_next_out to be this function at terminal time
    V[T][0] = V_terminal
    V[T][1] = V_terminal

    return V

def V_terminal(xT, tau, gamma):
    # Sell all assets and convert to cash in the final period
    return safe_utility(1 - tau * torch.sum(torch.abs(xT)), gamma)


# Sample state points function
def sample_state_points(D):
    points = []
    # Add corners of the simplex (ends)
    for i in range(2 ** D):
        point = [(i >> j) & 1 for j in range(D)]
        points.append(point)
    points.append([0] * D)
    # Add midpoints between all pairs of points
    for i in range(1, 2 ** D):
        for j in range(i):
            midpoint = [(a + b) / 2 for a, b in zip(points[i], points[j])]
            points.append(midpoint)
    # Add more midpoints by sampling regions with higher uncertainty (optional)
    points = [point for point in points if sum(point) <= 1]
    
    # Remove duplicates
    unique_points = []
    for point in points:
        if point not in unique_points:
            unique_points.append(point)
    
    return torch.tensor(unique_points, dtype=torch.float32)

# sample state points simplex
def sample_state_points_simplex(D, N):
    # Sample uniformly from the simplex
    def rand_simplex(n, k):
        # n: number of points, k: dimensions
        rand_points = np.random.dirichlet(np.ones(k), size=n)
        return rand_points

    points = rand_simplex(N, D)
    return torch.tensor(points, dtype=torch.float32)

def is_in_ntr(points, bound=0.005):
    # Calculate the sum of portfolio weights
    if points.dim() == 1:
        # If points is 1D, sum over the 0th dimension
        point_sums = torch.sum(points, dim=0)
    else:
        # If points is 2D, sum over the 1st dimension (each row)
        point_sums = torch.sum(points, dim=-1)    
    # Classify points as inside NTR if their sum is less than the bound
    ntr_mask = point_sums < bound
    
    return ntr_mask

def approximate_ntr(vertices):
    # Compute convex hull of the vertices to represent the NTR
    if len(vertices) > 2:  # Convex hull requires at least 3 points
        vertices = torch.stack(vertices).detach().numpy()  # Convert to numpy
        hull = ConvexHull(vertices)  # Compute convex hull
        return vertices, hull
    else:
        # Return the vertices directly if fewer than 3 points are available
        return vertices, None

def MertonPoint(mu, Sigma, r, gamma):
    # Step 1: Compute the diagonal matrix Lambda with sqrt of diagonal elements of Sigma
    Lambda = np.diag(np.sqrt(np.diag(Sigma)))
    
    # Step 2: Compute (Lambda * Sigma * Lambda)^(-1)
    Lambda_Sigma_Lambda = np.dot(Lambda, np.dot(Sigma, Lambda))
    Lambda_Sigma_Lambda_inv = np.linalg.inv(Lambda_Sigma_Lambda)
    
    # Step 3: Compute mu - r
    mu_r = mu - r
    
    # Step 4: Compute the Merton portfolio weights
    pi = np.dot(Lambda_Sigma_Lambda_inv, mu_r / gamma)
    
    return pi


# Bellman equation function
def bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf):

    # Compute bond holdings
    bt = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
    
    Rt = torch.tensor(mu,dtype=torch.float32)  # Simulated return
    
    # Compute next period wealth dynamics
    pi_t1, xt1 = normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf)

    # Compute utility from wealth
    u = safe_utility(pi_t1, gamma)
    
    # Determine whether the next state is inside or outside the NTR, and select the corresponding GPR
    if is_in_ntr(xt1):
        # Check if vt_next_in is a GP model or a function
        if isinstance(vt_next_in, gpytorch.models.ExactGP):
            vt_next_in.eval()  # Set the GP model to evaluation mode
            vt_next_val = vt_next_in(xt1.unsqueeze(0)).mean()
        elif callable(vt_next_in):  # If it's a function like V_terminal
            vt_next_val =  V_terminal(xt1, tau, gamma)
        else:
            raise TypeError("Expected vt_next_in to be a GP model or function.")
    else:
        # Check if vt_next_out is a GP model or a function
        if isinstance(vt_next_out, gpytorch.models.ExactGP):
            vt_next_out.eval()  # Set the GP model to evaluation mode
            vt_next_val = vt_next_out(xt1.unsqueeze(0)).mean()
        elif callable(vt_next_out):  # If it's a function like V_terminal
            vt_next_val =  V_terminal(xt1, tau, gamma)
        else:
            raise TypeError("Expected vt_next_out to be a GP model or function.")

    # Bellman equation
    vt = u + beta * torch.mean(pi_t1 ** (1 - gamma) * vt_next_val)
    # vt = beta * torch.mean(pi_t1 ** (1 - gamma) * vt_next_val)
    # vt = u + beta * pi_t1**(1-gamma) * vt_next_val

    return vt


# def solve_optimization(xt, vt_next_in, vt_next_out, t, T, D, beta, gamma, tau, Rf,mu,Sigma):
#     num_params = 2 * D

#     def objective(params):
#         delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
#         delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)
        
#         vt = bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf)
#         # return vt
#         return vt.item()  # Ensure returning a scalar    

#     def gradient(params):
#         delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
#         delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)

#         # Compute the value using the Bellman equation
#         vt = bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf)
        
#         # Backpropagate the gradients
#         vt.backward()

#         grad = np.concatenate([
#             delta_plus.grad.detach().numpy(),
#             delta_minus.grad.detach().numpy()
#         ])
        
#         return grad

#     # def constraints(params):
#     #     delta_plus = torch.tensor(params[:D], dtype=torch.float32,requires_grad=True)
#     #     delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32,requires_grad=True)
#     #     delta = torch.sum(delta_plus - delta_minus)
#     #     # Constraint 1: delta_plus >= 0
#     #     constraint_1 = delta_plus

#     #     # Constraint 2: delta_minus >= 0
#     #     constraint_2 = delta_minus


#     #     # Constraint 3: -delta_plus - delta_minus <= xt (can't sell more than you have)
#     #     constraint_3 = xt + delta_plus - delta_minus  # Final holdings should be non-negative
#     #     # # Constraint 4: bond holdings (b_t >= 0)
#     #     b_t = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     constraint_4 =  normalized_bond_holdings(xt, delta_plus, delta_minus, tau)

#     #     # Constraint 5: Total portfolio sum <= 1 (1^T * (xt + delta_plus - delta_minus) + b_t <= 1)
#     #     # total_sum = torch.sum(xt + delta_plus - delta_minus) + normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     # constraint_5 = 1.0 - torch.sum(xt + delta_plus - delta_minus) + normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     total_sum = torch.sum(xt + delta) + normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     constraint_5 = 1.0 - total_sum 
#     #     constraint_6 = total_sum - 0.0 # Ensure total sum is non-negative      
#     #     # Debugging print statements
#     #     print(f"Constraint 1 (delta_plus >= 0): {constraint_1}")
#     #     print(f"Constraint 2 (delta_minus >= 0): {constraint_2}")
#     #     print(f"Constraint 3 (delta <= xt): {constraint_3} with xt = {xt}")
#     #     print(f"Constraint 4 (b_t >= 0): {constraint_4}")
#     #     print(f"Constraint 5 (total portfolio sum <= 1): {constraint_5}")
#     #     print(f"bellman equation: {bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf)}")
        
#     #     # Combine all constraints into a tensor and return
#     #     constraints_combined = torch.cat([
#     #         constraint_1,  # delta_plus >= 0
#     #         constraint_2,  # delta_minus >= 0
#     #         constraint_3,  # delta_minus <= xt
#     #         constraint_4.unsqueeze(0),  # b_t >= 0
#     #         constraint_5.unsqueeze(0),   # total portfolio sum <= 1
#     #         constraint_6.unsqueeze(0)   # total portfolio sum >= 0
#     #     ])
#     #     return constraints_combined.detach().numpy()  # Return the constraints as a numpy array

#     # def constraints(params):
#     #     delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
#     #     delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)

#     #     b_t = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     constraint_1 = delta_plus
#     #     constraint_2 = delta_minus
#     #     constraint_3 = xt + delta_plus - delta_minus  # Final holdings should be non-negative

#     # #     # Constraint 5: Total portfolio sum <= 1 (1^T * (xt + delta_plus - delta_minus) + b_t <= 1)
#     #     # total_sum = torch.sum(xt + delta_plus - delta_minus) + normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     # constraint_5 = 1.0 - torch.sum(xt + delta_plus - delta_minus) + normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     # total_sum = torch.sum(xt + delta) + normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     constraint_4 = 1.0 - torch.sum(constraint_3) 

#     # #     # Debugging print statements
#     #     print(f"Constraint 1 {constraint_1}")
#     #     print(f"Constraint 2 {constraint_2}")
#     #     print(f"Constraint 3 {constraint_3} with xt = {xt}")
#     #     print(f"Constraint 4 {constraint_4}")
#     #     # print(f"Constraint 5 {constraint_5}")
#     #     print(f"bellman equation: {bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf)}")
        

#     #     # Combine all constraints into a tensor and return
#     #     constraints_combined = torch.cat([
#     #         constraint_1,  # delta_plus >= 0
#     #         constraint_2,  # delta_minus >= 0
#     #         constraint_3,  # delta_minus <= xt
#     #         b_t.unsqueeze(0),  # b_t >= 0
#     #         constraint_4.unsqueeze(0),   # total portfolio sum <= 1
#     #     ])
#     #     return constraints_combined.detach().numpy()  # Return as numpy arraynts_combined.detach().numpy()  # Return the constraints as numpy array


#     # # def jacobian(params):
#     # #     return np.eye(num_params)

#     # def jacobian(params):
#     #     constraints_combined = constraints(params)
#     #     jacobian_matrix = []

#     #     # Ensure that params is a torch tensor
#     #     params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)

#     #     for i in range(len(constraints_combined)):
#     #         # Convert constraint to a tensor if it's not already one
#     #         constraint_value = torch.tensor(constraints_combined[i], dtype=torch.float32, requires_grad=True)
            
#     #         # Compute gradient with allow_unused=True
#     #         grad = torch.autograd.grad(constraint_value, params_tensor, retain_graph=True, allow_unused=True)[0]
            
#     #         # If the gradient is None, replace it with zeros
#     #         if grad is None:
#     #             grad = torch.zeros_like(params_tensor)
            
#     #         jacobian_matrix.append(grad.numpy())

#     #     return np.array(jacobian_matrix)
    
#     # # Need some initial guesses
#     # initial_guesses = np.random.uniform(0, 0.1, size=2 * D)  # This should be a 1D array of size (4,)    # bounds = [(0, 1)] * D + [(0, 1)] * D  # Correct bounds for delta_plus and delta_minus
#     # # Use 0.1 as the initial guess for all params
#     # # initial_guesses = [np.full(num_params, 0.001) for _ in range(5)]
#     # bounds = [(0, 1)] * D + [(0, 1)] * D  # Correct bounds for delta_plus and delta_minus

#     # # result = minimize_ipopt(objective, initial_guesses, bounds=bounds, jac=gradient, 
#     # #                         constraints=[{'type': 'ineq', 'fun': constraints}],
#     # #                         options={'tol': 1e-5, 'maxiter': 100})
#     # result = minimize_ipopt(
#     #     fun=objective,
#     #     x0=initial_guesses,
#     #     jac=gradient,
#     #     bounds=bounds,
#     #     constraints=[{
#     #         'type': 'ineq',
#     #         'fun': constraints,
#     #         'jac': jacobian
#     #     }],    
#     #     options={'tol': 1e-5, 'maxiter': 1000},
#     #     # options={'maxiter': 100},
#     # )
#     # if result.success:
#     #     delta_plus = torch.tensor(result.x[:D], dtype=torch.float32)
#     #     delta_minus = torch.tensor(result.x[D:2*D], dtype=torch.float32)
#     #     omega_i_t = xt + delta_plus - delta_minus  # NTR vertice -> Point traded to

#     #     return delta_plus, delta_minus, omega_i_t
#     # else:
  
#     #     raise ValueError("Optimization did not converge.")


# def solve_optimization(xt, vt_next_in, vt_next_out, t, T, D, beta, gamma, tau, Rf,mu,Sigma):
#     # Define the number of decision variables (2D for portfolio choices + 1 for consumption only in final period)
#     # num_params = 2 * D + (1 if t == T else 0)
#     num_params = 2 * D

#     def objective(params):
#         delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
#         delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)
        
#         vt = bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf)
#         vt_minus = -vt
#         return vt_minus
#         # return vt_minus.item()


#     def gradient(params):
#         delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
#         delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)

#         # Compute the value using the Bellman equation
#         vt = bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf)
        
#         # Backpropagate the gradients
#         vt.backward()

#         grad = np.concatenate([
#             delta_plus.grad.detach().numpy(),
#             delta_minus.grad.detach().numpy()
#         ])
        
#         return grad

#     # def constraints(params):
#     #     delta_plus = torch.tensor(params[:D], dtype=torch.float32,requires_grad=True)
#     #     delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32,requires_grad=True)
#     #     delta = torch.sum(delta_plus - delta_minus)
#     #     # Constraint 1: delta_plus >= 0
#     #     constraint_1 = delta_plus
#     #     # Constraint 2: delta_minus >= 0
#     #     constraint_2 = delta_minus

#     #     # Constraint 3: -delta_plus - delta_minus <= xt (can't sell more than you have)
#     #     constraint_3 = xt + delta_plus - delta_minus  # Final holdings should be non-negative

#     #     # # Constraint 4: bond holdings (b_t >= 0)
#     #     b_t = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#     #     constraint_4 =  torch.tensor([b_t],dtype=torch.float32,requires_grad=True)

#     #     # Constraint 5: Total portfolio sum <= 1 (1^T * (xt + delta_plus - delta_minus) + b_t <= 1)
#     #     total_sum = torch.sum(xt + delta) + torch.tensor([b_t],dtype=torch.float32,requires_grad=True)
#     #     constraint_5 = torch.tensor([1.0],dtype=torch.float32) - total_sum 

#     #     # Constraint 6: Total portfolio sum >= 0
#     #     constraint_6 = total_sum 

#     #     # Debugging print statements
#     #     print(f"Constraint 1 (delta_plus >= 0): {constraint_1}")
#     #     print(f"Constraint 2 (delta_minus >= 0): {constraint_2}")
#     #     print(f"Constraint 3 (delta <= xt): {constraint_3} with xt = {xt}")
#     #     print(f"Constraint 4 (b_t >= 0): {constraint_4}")
#     #     print(f"Constraint 5 (total portfolio sum <= 1): {constraint_5}")
#     #     print(f"Constraint 6: {constraint_6}")
#     #     print(f"bellman equation: {bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf)}")
        
#     #     # Combine all constraints into a tensor and return
#     #     constraints_combined = torch.cat([
#     #         constraint_1,  # delta_plus >= 0
#     #         constraint_2,  # delta_minus >= 0
#     #         constraint_3,  # delta_minus <= xt
#     #         constraint_4,  # b_t >= 0
#     #         constraint_5,   # total portfolio sum <= 1
#     #         constraint_6   # total portfolio sum >= 0
#     #     ])
#     #     return constraints_combined.detach().numpy()  # Return the constraints as a numpy array


#     # def jacobian(params):
#     #     constraints_combined = constraints(params)
#     #     jacobian_matrix = []

#     #     # Ensure that params is a torch tensor
#     #     params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)

#     #     for i in range(len(constraints_combined)):
#     #         # Convert constraint to a tensor if it's not already one
#     #         constraint_value = torch.tensor(constraints_combined[i], dtype=torch.float32, requires_grad=True)
            
#     #         # Compute gradient with allow_unused=True
#     #         grad = torch.autograd.grad(constraint_value, params_tensor, retain_graph=True, allow_unused=True)[0]
            
#     #         # If the gradient is None, replace it with zeros
#     #         if grad is None:
#     #             grad = torch.zeros_like(params_tensor)
            
#     #         jacobian_matrix.append(grad.numpy())

#     #     return np.array(jacobian_matrix)

#     def constraints(params):
#         delta_plus = torch.tensor(params[:D], dtype=torch.float32)
#         delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32)
#         delta = delta_plus - delta_minus

#         # Constraint 1: delta_plus >= 0
#         constraint_1 = delta_plus

#         # Constraint 2: delta_minus >= 0
#         constraint_2 = delta_minus

#         # Constraint 3: Final holdings >= 0 (can't have negative asset positions)
#         constraint_3 = xt + delta

#         # Constraint 4: Bond holdings (b_t >= 0)
#         b_t = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#         constraint_4 = b_t

#         # Constraint 5: Total portfolio sum <= 1
#         total_sum = torch.sum(xt + delta) + b_t
#         constraint_5 = 1.0 - total_sum

#         # Constraint 6: Total portfolio sum >= 0 (non-negative wealth)
#         constraint_6 = total_sum

#         # Combine constraints into a single tensor
#         constraints_combined = torch.cat([
#             constraint_1,
#             constraint_2,
#             constraint_3,
#             constraint_4.unsqueeze(0),
#             constraint_5.unsqueeze(0),
#             constraint_6.unsqueeze(0)
#         ])

#         # Return constraints as a NumPy array
#         return constraints_combined.detach().numpy()

#     def jacobian(params):
#         # Convert params to a tensor with requires_grad=True
#         params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
#         delta_plus = params_tensor[:D]
#         delta_minus = params_tensor[D:2*D]
#         delta = delta_plus - delta_minus

#         # Recompute the constraints using tensors connected to params_tensor
#         constraint_1 = delta_plus
#         constraint_2 = delta_minus
#         constraint_3 = xt + delta
#         b_t = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)
#         constraint_4 = b_t
#         total_sum = torch.sum(xt + delta) + b_t
#         constraint_5 = 1.0 - total_sum
#         constraint_6 = total_sum

#         constraints_list = [
#             constraint_1,
#             constraint_2,
#             constraint_3,
#             constraint_4.unsqueeze(0),
#             constraint_5.unsqueeze(0),
#             constraint_6.unsqueeze(0)
#         ]

#         jacobian_matrix = []

#         # Compute gradients for each constraint
#         for constraint in constraints_list:
#             for c in constraint:
#                 grad = torch.autograd.grad(c, params_tensor, retain_graph=True)[0]
#                 jacobian_matrix.append(grad.detach().numpy())

#         return np.array(jacobian_matrix)    

#     # Use 0.1 as the initial guess for all params
#     initial_guesses = [np.full(num_params, 0.5) for _ in range(6)]
#     bounds = [(0, 1)] * D + [(0, 1)] * D  # Correct bounds for delta_plus and delta_minus

#     constraints_def = [{'type': 'ineq', 'fun': lambda x: constraints(x)}]
#     for initial_guess in initial_guesses:
#         # result = minimize_ipopt(objective, initial_guess, bounds=bounds, constraints=constraints_def, jac=gradient, options={'tol': 1e-6, 'maxiter': 1000})
#         result = minimize_ipopt(
#             fun=objective,
#             x0=initial_guess,
#             jac=gradient,
#             bounds=bounds,
#             get_current_iterate,
#             constraints=[{
#                 'type': 'ineq',
#                 'fun': constraints,
#                 'jac': jacobian
#             }],    
#             options={'tol': 1e-6, 'maxiter': 1000}
#             # options={'tol': 1e-6, 'maxiter': 1000, 'bound_relax_factor': 0}
#             ).add_option('bound_relax_factor', 0.0),

#         if result.success:
#             break

#     delta_plus = result.x[:D]
#     delta_minus = result.x[D:2*D]

#     delta_plus = torch.tensor(delta_plus,dtype=torch.float32,requires_grad=True)
#     delta_minus = torch.tensor(delta_minus,dtype=torch.float32,requires_grad=True)
#     omega_i_t = xt + delta_plus - delta_minus  # NTR vertice -> Point traded to
#     bt = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)

#     return delta_plus, delta_minus, delta_plus - delta_minus, omega_i_t, torch.tensor([1.0]) - torch.sum(omega_i_t) - torch.sum(bt)  # Return the optimal portfolio weights and the traded point



class PortfolioOptimization(cyipopt.Problem):
    def __init__(self, D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma):
        self.D = D
        self.xt = xt
        self.vt_next_in = vt_next_in
        self.vt_next_out = vt_next_out
        self.t = t
        self.T = T
        self.beta = beta
        self.gamma = gamma
        self.tau = tau
        self.Rf = Rf
        self.mu = mu
        self.Sigma = Sigma
        
        # Initialize the parent Problem class with dimensions
        n = 2 * D  # Number of variables: delta_plus and delta_minus
        m = 2 * D + 4  # Number of constraints

        super().__init__(n=n, m=m, lb=np.zeros(2 * D), ub=np.ones(2 * D),cl=np.zeros(m))  # lb and ub for decision variables

    def objective(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        delta_plus = params_tensor[:self.D]
        delta_minus = params_tensor[self.D:]

        vt = bellman_equation(self.vt_next_in, self.vt_next_out, self.xt, delta_plus, delta_minus, 
                            self.beta, self.gamma, self.tau, self.Rf)
        return -vt.item()

    def gradient(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        delta_plus = params_tensor[:self.D]
        delta_minus = params_tensor[self.D:]

        vt = bellman_equation(self.vt_next_in, self.vt_next_out, self.xt, delta_plus, delta_minus, 
                            self.beta, self.gamma, self.tau, self.Rf)
        
        # Backpropagate to compute gradients
        vt.backward()

        grad = params_tensor.grad.detach().numpy()
        
        return -grad  # Return negative gradient because of the negative objective

    def constraints(self, params):
        delta_plus = torch.tensor(params[:self.D], dtype=torch.float32)
        delta_minus = torch.tensor(params[self.D:], dtype=torch.float32)
        delta = delta_plus - delta_minus

        # Constraint 3: xt + delta >= 0 (no shorting or negative asset positions)
        constraint_3 = self.xt + delta

        # Constraint 5: Total portfolio weight sum <= 1
        total_sum = torch.sum(self.xt + delta)
        constraint_5 = 1.0 - total_sum  # Portfolio sum <= 1

        # Constraint 6: Total portfolio weight sum >= 0
        constraint_6 = total_sum

        # Return as a combined numpy array
        constraints_combined = torch.cat([
            constraint_3.view(-1),
            torch.tensor([constraint_5]),
            torch.tensor([constraint_6])
        ])
        return constraints_combined.detach().numpy()
    def jacobian(self, params):
        # Ensure that the params_tensor is created with requires_grad=True
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        delta_plus = params_tensor[:self.D]
        delta_minus = params_tensor[self.D:]
        delta = delta_plus - delta_minus

        # Recompute the constraints using tensors connected to params_tensor
        constraint_3 = self.xt + delta
        total_sum = torch.sum(self.xt + delta)
        constraint_5 = 1.0 - total_sum
        constraint_6 = total_sum

        constraints_list = [
            constraint_3.view(-1),
            constraint_5.view(-1),
            constraint_6.view(-1)
        ]

        jacobian_matrix = []

        # Compute gradients for each constraint
        for constraint in constraints_list:
            if constraint.dim() == 0:
                # Scalar constraint
                grad = torch.autograd.grad(constraint, params_tensor, retain_graph=True)[0]
                jacobian_matrix.append(grad.detach().numpy())
            else:
                # Iterable constraint
                for c in constraint:
                    grad = torch.autograd.grad(c, params_tensor, retain_graph=True)[0]
                    jacobian_matrix.append(grad.detach().numpy())

        # Flatten the Jacobian matrix
        jacobian_array = np.vstack(jacobian_matrix)
        return jacobian_array.flatten()

# Now set up and solve the optimization
def solve_optimization(D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma):
    # Set initial guess
    initial_guess = np.full(2 * D, 0.5)  # Set as needed

    # Create the optimization problem
    prob = PortfolioOptimization(D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma)

    # Set options if needed
    prob.add_option('tol', 1e-6)
    prob.add_option('max_iter', 1000)
    prob.add_option('bound_relax_factor', 0.0)
    prob.add_option('print_level', 5)
    # Solve the problem
    solution, info = prob.solve(initial_guess)

    return solution, info



# bellman_equation(V_terminal,V_terminal,torch.tensor([0.25,0.25]),torch.tensor([0.4,0.4]),torch.tensor([0.4,0.4]),beta,gamma,tau,Rf)
# solve_optimization(torch.tensor([0.5,0.4]),V_terminal,V_terminal,9,10,2,beta,gamma,tau,Rf,mu,Sigma)


# Define parameters and run the algorithm
# N = 250  # Number of sample points
# V,Ntr_test = dynamic_programming(T, N, D, gamma, beta, tau, Rf)

# Define your inputs
# xt = torch.tensor([0.5, 0.4],requires_grad=True)  # Initial portfolio weights
# vt_next_in = V_terminal  # Terminal value function for in-region
# vt_next_out = V_terminal  # Terminal value function for out-region
# t = 9  # Current time step

# # Call the solve_optimization function
# solution, info = solve_optimization(D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma)

# # Print the results
# print("Optimal Solution:", solution)
# print("Optimization Info:", info)

def approximate_ntr(vt_next_in, vt_next_out, D, t, T, beta, gamma, tau, Rf, mu, Sigma):
    # Step 1: Sample state points
    tilde_X_t = sample_state_points(D)
    N = len(tilde_X_t)
    tilde_omega_t = []

    for i in range(N):
        tilde_x_i_t = tilde_X_t[i]
        # Step 2: Solve optimization problem
        delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
            D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        )
        # Step 3: Compute NTR vertices
        tilde_omega_i_t = (tilde_x_i_t + delta).detach().numpy()
        tilde_omega_t.append(tilde_omega_i_t)

    # Step 4: Compute convex hull of the vertices to represent the NTR
    tilde_omega_t = np.array(tilde_omega_t)
    if len(tilde_omega_t) >= D + 1:
        convex_hull = ConvexHull(tilde_omega_t)
    else:
        convex_hull = None  # Cannot compute convex hull with fewer points

    return tilde_omega_t, convex_hull

def dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma):
    # Initialize value function V
    V = [[None, None] for _ in range(T + 1)]
    # Set terminal value function
    V[T][0] = lambda x: V_terminal(x, tau, gamma)
    V[T][1] = lambda x: V_terminal(x, tau, gamma)

    NTRs = [None for _ in range(T)]  # Store NTRs for each period

    for t in reversed(range(T)):
        print(f"Time step {t}")
        # Step 2a: Approximate NTR
        tilde_omega_t, convex_hull = approximate_ntr(V[t + 1][0], V[t + 1][1], D, t, T, beta, gamma, tau, Rf, mu, Sigma)
        NTRs[t] = convex_hull

        # Step 2b: Sample state points
        X_t = sample_state_points_simplex(D, N)
        data_in = []
        data_out = []

        for i in range(len(X_t)):
            x_i_t = X_t[i]
            # Step 2c: Solve optimization problem
            delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
                D, x_i_t, V[t + 1][0], V[t + 1][1], t + 1, T, beta, gamma, tau, Rf, mu, Sigma
            )
            # Compute value
            v_i_t = bellman_equation(V[t + 1][0], V[t + 1][1], x_i_t, delta_plus, delta_minus, beta, gamma, tau, Rf)
            # Determine if the point is inside the NTR
            x_i_t_np = x_i_t.detach().numpy()
            in_ntr = is_in_ntr(x_i_t_np, convex_hull)
            if in_ntr:
                data_in.append((x_i_t_np, v_i_t.detach().numpy()))
            else:
                data_out.append((x_i_t_np, v_i_t.detach().numpy()))

        # Step 2e: Train GPR models
        if data_in:
            train_x_in = torch.tensor([d[0] for d in data_in], dtype=torch.float32)
            train_y_in = torch.tensor([d[1] for d in data_in], dtype=torch.float32)
            model_in, likelihood_in = train_gp_model(train_x_in, train_y_in)
            V[t][0] = model_in
        else:
            V[t][0] = None

        if data_out:
            train_x_out = torch.tensor([d[0] for d in data_out], dtype=torch.float32)
            train_y_out = torch.tensor([d[1] for d in data_out], dtype=torch.float32)
            model_out, likelihood_out = train_gp_model(train_x_out, train_y_out)
            V[t][1] = model_out
        else:
            V[t][1] = None

    return V, NTRs



In [ ]:
xt = torch.tensor([0.5, 0.4],requires_grad=True)  # Initial portfolio weights
xt.view(-1)

In [ ]:
def plot_ntr_at_time(NTR_history, t):
    vertices, hull = NTR_history[t]
    if hull is not None:
        # Plot the convex hull
        plt.figure()
        for simplex in hull.simplices:
            plt.plot(vertices[simplex, 0], vertices[simplex, 1], 'k-')
        plt.fill(vertices[hull.vertices, 0], vertices[hull.vertices, 1], 'lightgray', alpha=0.4)
        plt.scatter(vertices[:, 0], vertices[:, 1], color='red')  # Plot the vertices
        plt.title(f'NTR at time {t}')
        plt.xlabel('State dimension 1')
        plt.ylabel('State dimension 2')
        # Set x and y axis limits
        plt.xlim(0, 1)
        plt.ylim(0, 1)        
        plt.show()
    else:
        print(f"Not enough vertices to form an NTR at time {t}")



# Example: Plot NTR at time t=3
plot_ntr_at_time(Ntr_test, 5)


In [ ]:
import numpy as np

import torch
import torch.autograd as autograd

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

import cyipopt
from cyipopt import Problem
from scipy.spatial import ConvexHull
from scipy.optimize import minimize
from numpy.polynomial.hermite import hermgauss

# Set random seed for reproducibility
np.random.seed(2001)
torch.manual_seed(2001)

# Parameters
T = 10  # Time horizon
D = 2  # Number of risky assets
r = 0.03  # Risk-free return in pct.
Rf = np.exp(r)  # Risk-free return
tau = 0.0025  # Transaction cost rate
beta = 0.975  # Discount factor
gamma = 3.0  # Risk aversion coefficient

# Risky assets - deterministic
mu = np.array([0.07, 0.07])
Sigma = np.array([[0.2, 0], [0, 0.2]])
Lambda = np.diag(np.sqrt(np.diag(Sigma)))

# Include consumption flag
include_consumption = False  # Set to True to include consumption


# Define the GPR model with ARD
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def train_gp_model(train_x, train_y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood(
        noise_constraint=gpytorch.constraints.GreaterThan(1e-6)
    )
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 200
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    return model, likelihood


def utility(var, gamma):
    if gamma == 1:
        return torch.log(var)  # Log utility for gamma = 1
    else:
        return (var ** (1.0 - gamma)) / (1 - gamma)  # CRRA utility


def safe_utility(var, gamma):
    var = torch.clamp(var, min=1e-10)
    return utility(var, gamma)


def normalized_bond_holdings(xt, delta_plus, delta_minus, tau, c_t=0.0):
    delta = delta_plus - delta_minus
    transaction_costs = tau * torch.sum(delta_plus - delta_minus)
    # Compute bond holdings
    bt = 1.0 - torch.sum(xt + delta) - transaction_costs - c_t
    bt = torch.clamp(bt, min=0.0)
    return bt


def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    delta = delta_plus - delta_minus
    # Wealth at t+1
    pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
    pi_t1 = torch.clamp(pi_t1, min=1e-10)  # Avoid division by zero or negative wealth

    # Portfolio weights at t+1
    xt1 = ((xt + delta) * Rt) / pi_t1
    xt1 = torch.clamp(xt1, min=0.0, max=1.0)  # Ensure valid weights
    return pi_t1, xt1

def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    delta = delta_plus - delta_minus
    # Wealth at t+1
    pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
    pi_t1 = torch.clamp(pi_t1, min=1e-10)  # Avoid division by zero or negative wealth
    # Portfolio weights at t+1
    xt1 = ((xt + delta) * Rt) / pi_t1
    # xt1 = torch.clamp(xt1, min=0.0, max=1.0)  # Ensure valid weights
    return pi_t1, xt1

def V_terminal(xT, tau, gamma):
    # Terminal utility function
    return safe_utility(1 - tau * torch.sum(torch.abs(xT)), gamma)

def bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf, c_t=0.0):
    # Compute bond holdings
    bt = normalized_bond_holdings(xt, delta_plus, delta_minus, tau, c_t)

    # Simulate returns (expected returns for simplicity)
    Rt = torch.tensor(mu, dtype=torch.float32)

    # Compute next period wealth dynamics
    pi_t1, xt1 = normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf)

    # Compute current utility
    u_t = safe_utility(pi_t1, gamma)

    # Determine whether the next state is inside or outside the NTR, and select the corresponding GPR
    if is_in_ntr(xt1):
        # Check if vt_next_in is a GP model or a function
        if isinstance(vt_next_in, gpytorch.models.ExactGP):
            vt_next_in.eval()  # Set the GP model to evaluation mode
            vt_next_val = vt_next_in(xt1.unsqueeze(0)).mean()
        elif callable(vt_next_in):  # If it's a function like V_terminal
            vt_next_val = vt_next_in(xt1, tau, gamma)
        else:
            raise TypeError("Expected vt_next_in to be a GP model or function.")
    else:
        # Check if vt_next_out is a GP model or a function
        if isinstance(vt_next_out, gpytorch.models.ExactGP):
            vt_next_out.eval()  # Set the GP model to evaluation mode
            vt_next_val = vt_next_out(xt1.unsqueeze(0)).mean()
        elif callable(vt_next_out):  # If it's a function like V_terminal
            vt_next_val = vt_next_out(xt1, tau, gamma)
        else:
            raise TypeError("Expected vt_next_out to be a GP model or function.")

    # Bellman equation
    vt = u_t + beta * vt_next_val

    return vt

def sample_state_points(D):
    points = []
    # Add corners of the simplex (ends)
    for i in range(2 ** D):
        point = [(i >> j) & 1 for j in range(D)]
        points.append(point)
    points.append([0] * D)
    # Add midpoints between all pairs of points
    for i in range(1, 2 ** D):
        for j in range(i):
            midpoint = [(a + b) / 2 for a, b in zip(points[i], points[j])]
            points.append(midpoint)
    # Add more midpoints by sampling regions with higher uncertainty (optional)
    points = [point for point in points if sum(point) <= 1]
    
    # Remove duplicates
    unique_points = []
    for point in points:
        if point not in unique_points:
            unique_points.append(point)
    
    return torch.tensor(unique_points, dtype=torch.float32)

# sample state points simplex
def sample_state_points_simplex(D, N):
    # Sample uniformly from the simplex
    def rand_simplex(n, k):
        # n: number of points, k: dimensions
        rand_points = np.random.dirichlet(np.ones(k), size=n)
        return rand_points

    points = rand_simplex(N, D)
    return torch.tensor(points, dtype=torch.float32)

def is_in_ntr(x, convex_hull):
    if convex_hull is None:
        return False

    new_point = np.array(x)
    hull = convex_hull
    A = hull.equations[:, :-1]
    b = -hull.equations[:, -1]
    inequalities = np.dot(A, new_point) + b
    return np.all(inequalities <= 1e-8)  # Allow for numerical tolerance

def approximate_ntr(vertices):
    # Compute convex hull of the vertices to represent the NTR
    if len(vertices) > 2:  # Convex hull requires at least 3 points
        vertices = torch.stack(vertices).detach().numpy()  # Convert to numpy
        hull = ConvexHull(vertices)  # Compute convex hull
        return vertices, hull
    else:
        # Return the vertices directly if fewer than 3 points are available
        return vertices, None

def MertonPoint(mu, Sigma, r, gamma):
    # Step 1: Compute the diagonal matrix Lambda with sqrt of diagonal elements of Sigma
    Lambda = np.diag(np.sqrt(np.diag(Sigma)))
    
    # Step 2: Compute (Lambda * Sigma * Lambda)^(-1)
    Lambda_Sigma_Lambda = np.dot(Lambda, np.dot(Sigma, Lambda))
    Lambda_Sigma_Lambda_inv = np.linalg.inv(Lambda_Sigma_Lambda)
    
    # Step 3: Compute mu - r
    mu_r = mu - r
    
    # Step 4: Compute the Merton portfolio weights
    pi = np.dot(Lambda_Sigma_Lambda_inv, mu_r / gamma)
    
    return pi


class PortfolioOptimization(Problem):
    def __init__(
        self,
        D,
        xt,
        vt_next_in,
        vt_next_out,
        t,
        T,
        beta,
        gamma,
        tau,
        Rf,
        mu,
        Sigma,
        include_consumption=False,
    ):
        self.D = D
        self.xt = xt
        self.vt_next_in = vt_next_in
        self.vt_next_out = vt_next_out
        self.t = t
        self.T = T
        self.beta = beta
        self.gamma = gamma
        self.tau = tau
        self.Rf = Rf
        self.mu = mu
        self.Sigma = Sigma
        self.include_consumption = include_consumption

        # Number of variables: c_t, delta_plus, delta_minus
        n = 1 + 2 * D  # Always include c_t

        # Number of constraints
        m = D + 3  # D constraints from xt + delta >= 0, and 3 scalar constraints

        # Variable bounds
        lb = np.zeros(n)
        ub = np.ones(n)
        if not self.include_consumption:
            # Fix c_t to zero
            lb[0] = 0.0
            ub[0] = 0.0  # Fix c_t to 0

        # Constraint bounds
        cl = np.zeros(m)
        cu = np.full(m, np.inf)  # All constraints are inequalities (>= 0)

        super().__init__(n=n, m=m, problem_obj=self, lb=lb, ub=ub, cl=cl, cu=cu)

    def objective(self, params):
        c_t = torch.tensor(params[0], dtype=torch.float32, requires_grad=True)
        idx = 1
        delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf,
            c_t,
        )
        # print(f"Objective function value: {-vt.item()}")

        if torch.isnan(vt).any() or torch.isinf(vt).any():
            raise ValueError("NaN or Inf detected in objective function!")

        return -vt.item()

    def gradient(self, params):
        c_t = torch.tensor(params[0], dtype=torch.float32, requires_grad=True)
        idx = 1
        delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf,
            c_t,
        )

        vt.backward()

        grads = []
        grads.append(c_t.grad.item())
        grads.extend(delta_plus.grad.detach().numpy())
        grads.extend(delta_minus.grad.detach().numpy())
        grads_array = np.array(grads)
        if np.isnan(grads_array).any() or np.isinf(grads_array).any():
            raise ValueError("NaN or Inf detected in gradients!")        

        return -np.array(grads)
    
    def compute_constraints(self, params_tensor):
        c_t = params_tensor[0]
        idx = 1
        delta_plus = params_tensor[idx : idx + self.D]
        delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
        delta = delta_plus - delta_minus

        # Compute constraints
        constraint_3 = self.xt + delta  # xt + delta >= 0
        total_sum = torch.sum(self.xt + delta)
        bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau, c_t)
        constraint_5 = 1.0 - (total_sum + bt)  # Total weights <= 1
        constraint_6 = total_sum + bt  # Total weights >= 0

        constraints_list = [
            constraint_3.view(-1),  # xt + delta constraints
            bt.view(-1),            # Bond holdings >= 0
            constraint_5.view(-1),  # Total weights <= 1
            constraint_6.view(-1),  # Total weights >= 0
        ]
        constraints_combined = torch.cat(constraints_list)
        return constraints_combined    

    def constraints(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)
        return constraints_combined.detach().numpy()

    def jacobian(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)

        jacobian_matrix = []

        # Compute gradients for each constraint
        for constraint in constraints_combined:
            grad = torch.autograd.grad(constraint, params_tensor, retain_graph=True)[0]
            jacobian_matrix.append(grad.detach().numpy())

        # Flatten the Jacobian matrix
        jacobian_array = np.vstack(jacobian_matrix)
        
        # Check for NaNs or Infs
        if np.isnan(jacobian_array).any() or np.isinf(jacobian_array).any():
            raise ValueError("NaN or Inf detected in Jacobian!")
        
        return jacobian_array.flatten()

    # def constraints(self, params):
    #     c_t = torch.tensor(params[0], dtype=torch.float32, requires_grad=True)
    #     idx = 1
    #     delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32)
    #     delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32)
    #     delta = delta_plus - delta_minus

    #     # print(f"delta_plus: {delta_plus}, delta_minus: {delta_minus}, delta: {delta}")

    #     constraints_list = []

    #     # Constraint 3: xt + delta >= 0 (no short positions)
    #     constraint_3 = self.xt + delta
    #     # print(f"xt + delta: {constraint_3}")

    #     constraints_list.append(constraint_3)

    #     # Constraint 4: bt >= 0
    #     bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau, c_t)
    #     # print(f"Bond holdings (bt): {bt}")
    #     constraints_list.append(bt.unsqueeze(0))

    #     # Constraint 5: Total portfolio weights <= 1
    #     total_weights = torch.sum(self.xt + delta) + bt
    #     # print(f"Total portfolio weights: {total_weights}")

    #     constraint_5 = 1.0 - total_weights
    #     constraints_list.append(constraint_5.unsqueeze(0))

    #     # Constraint 6: Total portfolio weights >= 0
    #     constraints_list.append(total_weights.unsqueeze(0))

    #     constraints_combined = torch.cat(constraints_list)
    #     # print(f"Constraints combined: {constraints_combined}")

    #     # Check if any constraints contain NaN or Inf
    #     if torch.isnan(constraints_combined).any() or torch.isinf(constraints_combined).any():
    #         raise ValueError("NaN or Inf detected in constraints!")

    #     return constraints_combined.detach().numpy()

    # def jacobian(self, params):
    #     # print("Jacobian is being evaluated.")

    #     # Ensure that params_tensor is created with requires_grad=True
    #     params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)

    #     c_t = params_tensor[0]
    #     idx = 1
    #     delta_plus = params_tensor[idx : idx + self.D]
    #     delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
    #     delta = delta_plus - delta_minus

    #     # Compute the constraints using tensors connected to params_tensor
    #     constraint_3 = self.xt + delta  # xt + delta >= 0 (no short positions)
    #     total_sum = torch.sum(self.xt + delta)
    #     bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau, c_t)
    #     constraint_5 = 1.0 - (total_sum + bt)  # Total weights <= 1
    #     constraint_6 = total_sum + bt  # Total weights >= 0

    #     constraints_list = [
    #         constraint_3.view(-1),  # xt + delta constraints
    #         bt.view(-1),            # Bond holdings >= 0
    #         constraint_5.view(-1),  # Total weights <= 1
    #         constraint_6.view(-1),  # Total weights >= 0
    #     ]
        
    #     jacobian_matrix = []

    #     # Compute gradients for each constraint
    #     for constraint in constraints_list:
    #         if constraint.dim() == 0:
    #             # Scalar constraint
    #             grad = torch.autograd.grad(constraint, params_tensor, retain_graph=True)[0]
    #             jacobian_matrix.append(grad.detach().numpy())
    #         else:
    #             # Vector constraint
    #             for c in constraint:
    #                 grad = torch.autograd.grad(c, params_tensor, retain_graph=True)[0]
    #                 jacobian_matrix.append(grad.detach().numpy())

    #     # Flatten the Jacobian matrix
    #     jacobian_array = np.vstack(jacobian_matrix)
    #     if np.isnan(jacobian_array).any() or np.isinf(jacobian_array).any():
    #         raise ValueError("NaN or Inf detected in Jacobian!")        
    #     # print("Jacobian matrix:\n", jacobian_array)
    #     return jacobian_array.flatten()

def solve_optimization(
    D,
    xt,
    vt_next_in,
    vt_next_out,
    t,
    T,
    beta,
    gamma,
    tau,
    Rf,
    mu,
    Sigma,
    include_consumption=False,
):
    # Number of variables
    n = 1 + 2 * D  # Always include c_t

    # Initial guess
    initial_guess = np.full(n, 0.01)
    if not include_consumption:
        initial_guess[0] = 0.001  # Set c_t initial guess to zero

    # Create the optimization problem
    prob = PortfolioOptimization(
        D,
        xt,
        vt_next_in,
        vt_next_out,
        t,
        T,
        beta,
        gamma,
        tau,
        Rf,
        mu,
        Sigma,
        include_consumption=include_consumption,
    )

    # Set options if needed
    prob.add_option("tol", 1e-5)
    prob.add_option("max_iter", 500)
    prob.add_option("print_level", 5)
    prob.add_option("check_derivatives_for_naninf","yes")
    prob.add_option("derivative_test", "first-order")
    prob.add_option("derivative_test_tol", 1e-6)
    # Solve the problem
    solution, info = prob.solve(initial_guess)

    return solution, info

# Example usage
if __name__ == "__main__":
    # Initial portfolio weights
    xt = torch.tensor([0.4, 0.4])
    # Terminal value functions (using V_terminal)
    vt_next_in = V_terminal
    vt_next_out = V_terminal
    t = 9  # Current time step

    # Solve the optimization problem
    solution, info = solve_optimization(
        D,
        xt,
        vt_next_in,
        vt_next_out,
        t,
        T,
        beta,
        gamma,
        tau,
        Rf,
        mu,
        Sigma,
        include_consumption=include_consumption,
    )

    # Extract results
    c_t_opt = solution[0]
    idx = 1
    delta_plus_opt = solution[idx : idx + D]
    delta_minus_opt = solution[idx + D : idx + 2 * D]
    delta_opt = delta_plus_opt - delta_minus_opt

    print("Optimal Consumption (c_t):", np.round(c_t_opt,4))
    print("Optimal Delta Plus:", np.round(delta_plus_opt,4))
    print("Optimal Delta Minus:", np.round(delta_minus_opt,4))
    print("Optimal Portfolio Change (delta):", np.round(delta_opt,4))
    print("New Portfolio Weights:", np.round(xt.numpy() + delta_opt,4))
    # print("Optimization Info:", info)

# N = 100  # Number of points to sample

# # Sample N points from the simplex
# X_t = sample_state_points_simplex(D, N)

# # For each point in X_t, solve the optimization problem
# results = []
# for x_t in X_t:
#     solution, info = solve_optimization(
#         D,
#         x_t,
#         vt_next_in,
#         vt_next_out,
#         t,
#         T,
#         beta,
#         gamma,
#         tau,
#         Rf,
#         mu,
#         Sigma,
#         include_consumption=include_consumption,
#     )
#     # Extract the value function approximation
#     v_t_approx = -info['obj_val']  # Since we minimized -vt
#     results.append((x_t, v_t_approx))
    
from scipy.spatial import ConvexHull

def approximate_ntr(vt_next_in, vt_next_out, D, t, T, beta, gamma, tau, Rf, mu, Sigma):
    # Step 1: Sample state points
    tilde_X_t = sample_state_points(D)
    N = len(tilde_X_t)
    tilde_omega_t = []

    for i in range(N):
        tilde_x_i_t = tilde_X_t[i]
        # Step 2: Solve optimization problem
        delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
            D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        )
        # Step 3: Compute NTR vertices
        tilde_omega_i_t = (tilde_x_i_t + delta).detach().numpy()
        tilde_omega_t.append(tilde_omega_i_t)

    # Step 4: Compute convex hull of the vertices to represent the NTR
    tilde_omega_t = np.array(tilde_omega_t)
    if len(tilde_omega_t) >= D + 1:
        convex_hull = ConvexHull(tilde_omega_t)
    else:
        convex_hull = None  # Cannot compute convex hull with fewer points

    return tilde_omega_t, convex_hull

def dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma):
    # Initialize value function V
    V = [[None, None] for _ in range(T + 1)]
    # Set terminal value function
    V[T][0] = lambda x: V_terminal(x, tau, gamma)
    V[T][1] = lambda x: V_terminal(x, tau, gamma)

    NTRs = [None for _ in range(T)]  # Store NTRs for each period

    for t in reversed(range(T)):
        print(f"Time step {t}")
        # Step 2a: Approximate NTR
        tilde_omega_t, convex_hull = approximate_ntr(V[t + 1][0], V[t + 1][1], D, t, T, beta, gamma, tau, Rf, mu, Sigma)
        NTRs[t] = convex_hull

        # Step 2b: Sample state points
        X_t = sample_state_points_simplex(D, N)
        data_in = []
        data_out = []

        for i in range(len(X_t)):
            x_i_t = X_t[i]
            # Step 2c: Solve optimization problem
            delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
                D, x_i_t, V[t + 1][0], V[t + 1][1], t + 1, T, beta, gamma, tau, Rf, mu, Sigma
            )
            # Compute value
            v_i_t = bellman_equation(V[t + 1][0], V[t + 1][1], x_i_t, delta_plus, delta_minus, beta, gamma, tau, Rf)
            # Determine if the point is inside the NTR
            x_i_t_np = x_i_t.detach().numpy()
            in_ntr = is_in_ntr(x_i_t_np, convex_hull)
            if in_ntr:
                data_in.append((x_i_t_np, v_i_t.detach().numpy()))
            else:
                data_out.append((x_i_t_np, v_i_t.detach().numpy()))

        # Step 2e: Train GPR models
        if data_in:
            train_x_in = torch.tensor([d[0] for d in data_in], dtype=torch.float32)
            train_y_in = torch.tensor([d[1] for d in data_in], dtype=torch.float32)
            model_in, likelihood_in = train_gp_model(train_x_in, train_y_in)
            V[t][0] = model_in
        else:
            V[t][0] = None

        if data_out:
            train_x_out = torch.tensor([d[0] for d in data_out], dtype=torch.float32)
            train_y_out = torch.tensor([d[1] for d in data_out], dtype=torch.float32)
            model_out, likelihood_out = train_gp_model(train_x_out, train_y_out)
            V[t][1] = model_out
        else:
            V[t][1] = None

    return V, NTRs    

In [ ]:
results

# Current implementation

In [ ]:
import numpy as np

import torch
import torch.autograd as autograd

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

import cyipopt
from cyipopt import Problem
from scipy.spatial import ConvexHull
from scipy.optimize import minimize
from numpy.polynomial.hermite import hermgauss

import logging

# Set up logging configuration
logging.basicConfig(filename='optimization_log.txt', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')


# Set random seed for reproducibility
np.random.seed(2001)
torch.manual_seed(2001)

# Parameters
T = 10  # Time horizon
D = 2  # Number of risky assets
r = 0.02  # Risk-free return in pct.
Rf = np.exp(r)  # Risk-free return
Rf = r  # Risk-free return
tau = 0.005  # Transaction cost rate
# tau = 0.0
beta = 0.975  # Discount factor
gamma = 3.0  # Risk aversion coefficient

# Risky assets - deterministic
mu = np.array([0.07, 0.07])
Sigma = np.array([[0.2, 0], [0, 0.2]])
Lambda = np.diag(np.sqrt(np.diag(Sigma)))

# Include consumption flag
include_consumption = False  # Set to True to include consumption


# Define the GPR model with ARD
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def train_gp_model(train_x, train_y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood(
        noise_constraint=gpytorch.constraints.GreaterThan(1e-6)
    )
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()

    # optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 350
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    return model, likelihood


def utility(var, gamma):
    if gamma == 1:
        return torch.log(var)  # Log utility for gamma = 1
    else:
        return (var ** (1.0 - gamma)) / (1 - gamma)  # CRRA utility


def safe_utility(var, gamma):
    var = torch.clamp(var, min=1e-10)
    return utility(var, gamma)


def normalized_bond_holdings(xt, delta_plus, delta_minus, tau, c_t=0.0):
    delta_plus = torch.tensor(delta_plus,dtype=torch.float32,requires_grad=True)
    delta_minus = torch.tensor(delta_minus,dtype=torch.float32,requires_grad=True)

    delta = delta_plus - delta_minus
    transaction_costs = tau * torch.sum(delta_plus - delta_minus)
    # Compute bond holdings
    bt = 1.0 - torch.sum(xt + delta) - transaction_costs - c_t
    # bt = torch.clamp(bt, min=0.0)
    return bt



def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    delta = delta_plus - delta_minus
    # Wealth at t+1
    pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
    # pi_t1 = torch.clamp(pi_t1, min=1e-10)  # Avoid division by zero or negative wealth
    # Portfolio weights at t+1
    xt1 = ((xt + delta) * Rt) / pi_t1
    # xt1 = torch.clamp(xt1, min=0.0, max=1.0)  # Ensure valid weights
    return pi_t1, xt1

def V_terminal(xT):
    return utility(1.0 - tau * torch.sum(torch.abs(xT)), gamma)

def bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf, c_t=0.0,convex_hull=None):
    xt = avoid_zero_points(xt)
    
    # Compute bond holdings
    bt = normalized_bond_holdings(xt, delta_plus, delta_minus, tau, c_t)

    # Simulate returns (expected returns for simplicity)
    Rt = torch.tensor(mu, dtype=torch.float32)

    # Compute next period wealth dynamics
    pi_t1, xt1 = normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf)

    # Determine whether the next state is inside or outside the NTR
    xt1_np = xt1.detach().numpy()

    if torch.is_tensor(vt_next_in):
        print("vt_next_in is a tensor")
    if torch.is_tensor(vt_next_out):
        print("vt_next_in is a tensor")

    if is_in_ntr(xt1_np, convex_hull):
        # Inside the NTR, use vt_next_in
        if isinstance(vt_next_in, gpytorch.models.ExactGP):
            vt_next_in.eval()
            vt_next_val = vt_next_in(torch.tensor(xt1_np, dtype=torch.float32).unsqueeze(0)).mean()
        elif callable(vt_next_in):
            vt_next_val = vt_next_in(xt1)
        elif vt_next_in is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_in to be a GP model or function.")
    else:
        # Outside the NTR, use vt_next_out
        if isinstance(vt_next_out, gpytorch.models.ExactGP):
            vt_next_out.eval()
            vt_next_val = vt_next_out(torch.tensor(xt1_np, dtype=torch.float32).unsqueeze(0)).mean()
        elif callable(vt_next_out):
            vt_next_val = vt_next_out(xt1)
        elif vt_next_out is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_out to be a GP model or function.")

    # Compute the value function
    # vt = safe_utility(pi_t1, gamma) + beta * torch.mean(pi_t1 ** (1.0 - gamma) * vt_next_val)
    vt = beta * torch.mean(pi_t1 ** (1.0 - gamma) * vt_next_val)

    return vt

def sample_state_points(D):
    small_value = 1e-8
    points = []
    # Add corners of the simplex (ends)
    for i in range(2 ** D):
        # point = [(i >> j) & 1 for j in range(D)]
        point = [(1 - small_value) if ((i >> j) & 1) else small_value for j in range(D)]
        points.append(point)
    # points.append([0] * D)
    # Avoid adding the point [0, 0] by perturbing it
    points.append([small_value] * D)    
    # Add midpoints between all pairs of points
    for i in range(1, 2 ** D):
        for j in range(i):
            midpoint = [(a + b) / 2 for a, b in zip(points[i], points[j])]
            points.append(midpoint)
    # Add more midpoints by sampling regions with higher uncertainty (optional)
    points = [point for point in points if sum(point) <= 1]
    
    # Remove duplicates
    unique_points = []
    for point in points:
        if point not in unique_points:
            unique_points.append(point)
    
    return torch.tensor(unique_points, dtype=torch.float32)

# Sample state points from the simplex
def sample_state_points_simplex(D, N):
    # Generate a grid of points in the simplex
    def grid_simplex(n, k):
        # n: number of points, k: dimensions
        grid_points = []
        for _ in range(n):
            point = np.random.rand(k)
            point /= np.sum(point)
            grid_points.append(point)
        return np.array(grid_points)

    points = grid_simplex(N, D)
    points = np.maximum(points, 1e-8)
    return torch.tensor(points, dtype=torch.float32)

def avoid_zero_points(xt):
    return torch.clamp(xt, min=1e-8)  # Replace exact zero with small perturbation

def is_in_ntr(x, convex_hull):
    if convex_hull is None:
        return False

    new_point = np.array(x)
    hull = convex_hull
    A = hull.equations[:, :-1]
    b = -hull.equations[:, -1]
    inequalities = np.dot(A, new_point) + b
    return np.all(inequalities <= 1e-8)  # Allow for numerical tolerance

def approximate_ntr(vertices):
    # Compute convex hull of the vertices to represent the NTR
    if len(vertices) > 2:  # Convex hull requires at least 3 points
        vertices = torch.stack(vertices).detach().numpy()  # Convert to numpy
        hull = ConvexHull(vertices)  # Compute convex hull
        return vertices, hull
    else:
        # Return the vertices directly if fewer than 3 points are available
        return vertices, None

def MertonPoint(mu, Sigma, r, gamma):
    # Step 1: Compute the diagonal matrix Lambda with sqrt of diagonal elements of Sigma
    Lambda = np.diag(np.sqrt(np.diag(Sigma)))
    
    # Step 2: Compute (Lambda * Sigma * Lambda)^(-1)
    Lambda_Sigma_Lambda = np.dot(Lambda, np.dot(Sigma, Lambda))
    Lambda_Sigma_Lambda_inv = np.linalg.inv(Lambda_Sigma_Lambda)
    
    # Step 3: Compute mu - r
    mu_r = mu - r
    
    # Step 4: Compute the Merton portfolio weights
    pi = np.dot(Lambda_Sigma_Lambda_inv, mu_r / gamma)
    
    return pi


class PortfolioOptimization(Problem):
    def __init__(
        self,
        D,
        xt,
        vt_next_in,
        vt_next_out,
        t,
        T,
        beta,
        gamma,
        tau,
        Rf,
        mu,
        Sigma,
        include_consumption=False,
    ):
        self.D = D
        self.xt = xt
        self.vt_next_in = vt_next_in
        self.vt_next_out = vt_next_out
        self.t = t
        self.T = T
        self.beta = beta
        self.gamma = gamma
        self.tau = tau
        self.Rf = Rf
        self.mu = mu
        self.Sigma = Sigma
        self.include_consumption = include_consumption

        # Number of variables: c_t, delta_plus, delta_minus
        n = 1 + 2 * D  # Always include c_t

        # Number of constraints
        # m = 4*D + 3  # D constraints from xt + delta >= 0, and 3 scalar constraints
        m = D + 3  # D constraints from xt + delta >= 0, and 3 scalar constraints

        # Variable bounds
        lb = np.zeros(n)
        ub = np.ones(n)
        if not self.include_consumption:
            # Fix c_t to zero
            lb[0] = 0.0
            ub[0] = 0.0  # Fix c_t to 0

        # Constraint bounds
        # cl = np.zeros(m)
        cl = np.zeros(m)
        cu = np.full(m, np.inf)  # All constraints are inequalities (>= 0)

        super().__init__(n=n, m=m, problem_obj=self, lb=lb, ub=ub, cl=cl, cu=cu)

    def objective(self, params):
        c_t = torch.tensor(params[0], dtype=torch.float32, requires_grad=True)
        idx = 1
        delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf,
            c_t,
        )
        
        if torch.isnan(vt).any() or torch.isinf(vt).any():
            raise ValueError("NaN or Inf detected in objective function!")

        # return -vt.item()
        # print(f"objective: {vt.item()}")
        return vt.item() #TODO

    def gradient(self, params):
        c_t = torch.tensor(params[0], dtype=torch.float32, requires_grad=True)
        idx = 1
        delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf,
            c_t,
        )

        vt.backward()

        grads = []
        grads.append(c_t.grad.item())
        grads.extend(delta_plus.grad.detach().numpy())
        grads.extend(delta_minus.grad.detach().numpy())
        grads_array = np.array(grads)
        if np.isnan(grads_array).any() or np.isinf(grads_array).any():
            print(f"gradient: {grads_array}")
            print(f"params: {params}")
            print(f"vt: {vt}")
            print(f"c_t: {c_t}")
            print(f"delta_plus: {delta_plus}")
            print(f"delta_minus: {delta_minus}")
            print(f"x_t: {self.xt}")
            print(f"vt_next_in: {self.vt_next_in}")
            print(f"vt_next_out: {self.vt_next_out}")   
            raise ValueError("NaN or Inf detected in gradients!")     

        # return -np.array(grads) #TODO
        return np.array(grads)
    
    def compute_constraints(self, params_tensor):
        c_t = params_tensor[0]
        idx = 1
        delta_plus = params_tensor[idx : idx + self.D]
        delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
        delta = delta_plus - delta_minus

        # Compute constraints
        constraint_3 = self.xt + delta  # xt + delta >= 0
        total_sum = torch.sum(self.xt + delta)
        bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau, c_t)
        constraint_5 = 1.0 - (total_sum + bt)  # Total weights <= 1
        constraint_6 = total_sum + bt  # Total weights >= 0

        constraints_list = [
            constraint_3.view(-1),  # xt + delta constraints
            bt.view(-1),            # Bond holdings >= 0
            constraint_5.view(-1),  # Total weights <= 1
            constraint_6.view(-1),  # Total weights >= 0
        ]
        constraints_combined = torch.cat(constraints_list)
        return constraints_combined    

    def constraints(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)
        return constraints_combined.detach().numpy()

    def jacobian(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)

        jacobian_matrix = []

        # Compute gradients for each constraint
        for constraint in constraints_combined:
            grad = torch.autograd.grad(constraint, params_tensor, retain_graph=True)[0]
            jacobian_matrix.append(grad.detach().numpy())

        # Flatten the Jacobian matrix
        jacobian_array = np.vstack(jacobian_matrix)
        
        # Check for NaNs or Infs
        if np.isnan(jacobian_array).any() or np.isinf(jacobian_array).any():
            raise ValueError("NaN or Inf detected in Jacobian!")
        
        return jacobian_array.flatten()

def solve_bellman_with_ipopt(
    D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma,
    include_consumption=False, num_starts=7, drop_tolerance=0.2
):
    best_solution = None
    best_info = None
    best_obj_val = float('-inf')
    failed_attempts = 0
    max_failed_attempts = int(num_starts * drop_tolerance)

    def generate_feasible_initial_guess(X_t, D, tau, include_consumption=False):
        # Ensure that delta_plus and delta_minus satisfy the constraints
        delta_plus = np.zeros(D)
        delta_minus = np.zeros(D)

        total_available = 1.0 - np.sum(X_t)  # Total available room for adjustments

        for d in range(D):
            # Compute the available room for delta_plus and delta_minus based on X_t
            # Limit delta_plus to be within remaining available space and current asset holding
            max_delta_plus = min(X_t[d], total_available)

            # Randomly assign delta_plus within the available range
            delta_plus[d] = np.random.uniform(0, max_delta_plus)

            # Calculate the available room for delta_minus given delta_plus
            max_delta_minus = X_t[d] - delta_plus[d]
            delta_minus[d] = np.random.uniform(0, max_delta_minus)

            # Update the total available room for the next assets
            total_available -= delta_plus[d]

        # Compute transaction costs
        transaction_costs = tau * np.sum(delta_plus - delta_minus)

        # Compute bond holdings (bt), ensuring non-negative bond holdings
        bt = 1.0 - np.sum(X_t + delta_plus - delta_minus) - transaction_costs
        if bt < 0:
            raise ValueError("Initial guess led to infeasible bond holdings. This should not happen!")

        # Optionally include consumption
        c_t = 0.0 if not include_consumption else np.random.uniform(0, 0.05)

        return np.concatenate([[c_t], delta_plus, delta_minus])
    # Loop through multiple starting points (up to 5)
    for start_idx in range(num_starts):
        initial_guess = np.random.uniform(0, 0.9, size=1 + 2 * D)  # Random initial guess
        initial_guess = generate_feasible_initial_guess(xt.detach().numpy(), D, tau, include_consumption)
        # if not include_consumption:
        #     initial_guess[0] = 0.0  # Set c_t to zero if no consumption

        try:
            # Create the optimization problem
            print(f"time: {t} point: {xt}, start: {initial_guess}") 
            prob = PortfolioOptimization(
                D,
                xt,
                vt_next_in,
                vt_next_out,
                t,
                T,
                beta,
                gamma,
                tau,
                Rf,
                mu,
                Sigma,
                include_consumption=include_consumption,
            )


            prob.add_option("tol", 1e-6)
            prob.add_option("max_iter", 300)
            prob.add_option("acceptable_tol", 1e-5)
            prob.add_option("print_level", 4)
            prob.add_option("check_derivatives_for_naninf","yes")
            prob.add_option("derivative_test", "first-order")
            prob.add_option("derivative_test_tol", 1e-6)
            prob.add_option("mu_strategy", "adaptive")  # Adaptive step size strategy
            prob.add_option("mu_oracle", "quality-function")  # Control step quality
            prob.add_option("honor_original_bounds", "yes")
            prob.add_option("barrier_tol_factor", 1e-5)



            solution, info = prob.solve(initial_guess)

            # Check if this solution is better than the current best
            if info['status'] == 0 and (best_solution is None or info['obj_val'] > best_obj_val):
                best_solution = solution
                best_info = info
                best_obj_val = info['obj_val']

        except Exception as e:
            print(f"Optimization failed for start {start_idx}: {e}")
            failed_attempts += 1
            # If too many failures occur, drop this point
            if failed_attempts > max_failed_attempts:
                print(f"Exceeded maximum allowed failed attempts: {max_failed_attempts}")
                return None, None
            continue

    if best_solution is None:
        bt = normalized_bond_holdings(xt, torch.zeros(D), torch.zeros(D), tau).item()
        print(f"Xt: {xt}")
        print(f"vt_next_in: {vt_next_in}") 
        print(f"vt_next_out: {vt_next_out}")
        print(f"t: {t}")
        print(f"T: {T}")
        print(f"bt: {bt}")
        raise ValueError("All initial guesses led to infeasibility.")

    # After finding the best solution, extract the variables
    c_t_opt = best_solution[0]
    idx = 1
    delta_plus_opt = best_solution[idx : idx + D]
    delta_minus_opt = best_solution[idx + D : idx + 2 * D]
    delta_opt = delta_plus_opt - delta_minus_opt

    # Compute omega_i_t and bond holdings (bt)
    omega_i_t = xt.detach().numpy() + delta_opt
    bt = normalized_bond_holdings(
        xt, torch.tensor(delta_plus_opt), torch.tensor(delta_minus_opt), tau, c_t_opt
    ).item()

    return delta_plus_opt, delta_minus_opt, delta_opt, omega_i_t, bt


def approximate_ntr(vt_next_in, vt_next_out, D, t, T, beta, gamma, tau, Rf, mu, Sigma):
    # Step 1: Sample state points
    tilde_X_t = sample_state_points(D)
    N = len(tilde_X_t)
    tilde_omega_t = []

    for i in range(N):
        tilde_x_i_t = tilde_X_t[i]
        # Step 2: Solve optimization problem
        # delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
        #     D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        # )
        delta_plus, delta_minus, delta, omega_i_t, b_t = solve_bellman_with_ipopt(
            D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        )
        # Step 3: Compute NTR vertices
        tilde_omega_i_t = (tilde_x_i_t + delta).detach().numpy()
        tilde_omega_t.append(tilde_omega_i_t)

    # Step 4: Compute convex hull of the vertices to represent the NTR
    tilde_omega_t = np.array(tilde_omega_t)
    if len(tilde_omega_t) >= D + 1:
        convex_hull = ConvexHull(tilde_omega_t)
    else:
        convex_hull = None  # Cannot compute convex hull with fewer points

    return tilde_omega_t, convex_hull

def dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma):
    # Initialize value function V
    V = [[None, None] for _ in range(T + 1)]
    
    # Set terminal value function
    V[T][0] = V_terminal  # For inside NTR
    V[T][1] = V_terminal  # For outside NTR

    NTRs = [None for _ in range(T)]  # Store NTRs for each period

    for t in reversed(range(T)):
        print(f"Time step {t}")

        # Step 2a: Approximate NTR
        tilde_omega_t, convex_hull = approximate_ntr(V[t + 1][0], V[t + 1][1], D, t, T, beta, gamma, tau, Rf, mu, Sigma)
        NTRs[t] = convex_hull

        # Step 2b: Sample state points
        X_t = sample_state_points_simplex(D, N)
        data_in = []
        data_out = []

        for i in range(len(X_t)):
            x_i_t = X_t[i]
            # Step 2c: Solve optimization problem
            # delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
            #     D, x_i_t, V[t + 1][0], V[t + 1][1], t + 1, T, beta, gamma, tau, Rf, mu, Sigma
            # )
            delta_plus, delta_minus, delta, omega_i_t, b_t = solve_bellman_with_ipopt(
                D, x_i_t, V[t + 1][0], V[t + 1][1], t, T, beta, gamma, tau, Rf, mu, Sigma
            )
            
            # Compute value using Bellman equation, selecting the correct V[t+1]
            v_i_t = bellman_equation(V[t + 1][0], V[t + 1][1], x_i_t, delta_plus, delta_minus, beta, gamma, tau, Rf)

            # Determine if the point is inside the NTR and append to the respective data set
            x_i_t_np = x_i_t.detach().numpy()
            in_ntr = is_in_ntr(x_i_t_np, convex_hull)
            if in_ntr:
                data_in.append((x_i_t_np, v_i_t.detach().numpy()))  # Ensure numpy values
            else:
                data_out.append((x_i_t_np, v_i_t.detach().numpy()))  # Ensure numpy values
            print(x_i_t , delta_plus, delta_minus, delta, omega_i_t)

        # # Step 2e: Train GPR models for inside and outside NTR
        if data_in:
            # Print for debugging
            print("Data out structure:", [(d[0], d[1].shape if isinstance(d[1], torch.Tensor) else "scalar") for d in data_in])
            
            try:
                # Convert list of data_out values to tensors
                train_x_in = torch.tensor([d[0] for d in data_in], dtype=torch.float32)
                
                # Handle scalar values by converting them into 1D tensors with unsqueeze
                train_y_in = torch.tensor([d[1].item() if isinstance(d[1], torch.Tensor) else d[1] for d in data_in], dtype=torch.float32).unsqueeze(1)

                # Ensure that train_y_out is the correct shape
                model_in, likelihood_in = train_gp_model(train_x_in,train_y_in)

                if model_in is not None:
                    V[t][0] = model_in  # Surrogate for outside NTR
            except Exception as e:
                print("Error in constructing train_y_in:", e)
        else:
            V[t][1] = V_terminal  # Fallback if no outside NTR data     

        # Step 2e: Train GPR models for inside and outside NTR
        if data_out:
            try:
                # Print for debugging
                print("Data out structure:", [(d[0], d[1].shape if isinstance(d[1], torch.Tensor) else "scalar") for d in data_out])

                # Convert list of data_out values to tensors
                train_x_out = torch.tensor([d[0] for d in data_out], dtype=torch.float32)

                # Handle scalar values and wrap them into 1D tensors with unsqueeze
                train_y_out = torch.tensor([d[1] for d in data_out], dtype=torch.float32).unsqueeze(1)

                # Train GP model
                model_out, likelihood_out = train_gp_model(train_x_out, train_y_out)

                if model_out is not None:
                    V[t][1] = model_out  # Surrogate for outside NTR
                else:
                    V[t][1] = V_terminal
            except Exception as e:
                print("Error in constructing train_y_out:", e)
                print(f"type(data_out): {type(data_out)}")
        else:
            V[t][1] = V_terminal  # Fallback if no outside NTR data
        
    return V, NTRs

# Parameters
T = 6  # Time horizon
N = 100  # Number of sample points
D = 2  # Number of risky assets

# Run the dynamic programming algorithm
V, NTRs = dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma)

# REMOVED CONSUMPTION

In [ ]:
import numpy as np

import torch
import torch.autograd as autograd

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

import cyipopt
from cyipopt import Problem
from scipy.spatial import ConvexHull
from scipy.optimize import minimize
from numpy.polynomial.hermite import hermgauss

import logging

# Set up logging configuration
logging.basicConfig(filename='optimization_log.txt', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')


# Set random seed for reproducibility
np.random.seed(2001)
torch.manual_seed(2001)

# Parameters
T = 10  # Time horizon
D = 2  # Number of risky assets
r = 0.02  # Risk-free return in pct.
Rf = np.exp(r)  # Risk-free return
Rf = r  # Risk-free return
tau = 0.005  # Transaction cost rate
# tau = 0.0
beta = 0.975  # Discount factor
gamma = 3.0  # Risk aversion coefficient

# Risky assets - deterministic
mu = np.array([0.07, 0.07])
Sigma = np.array([[0.2, 0], [0, 0.2]])
Lambda = np.diag(np.sqrt(np.diag(Sigma)))

# Include consumption flag
include_consumption = False  # Set to True to include consumption


# Define the GPR model with ARD
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def train_gp_model(train_x, train_y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood(
        noise_constraint=gpytorch.constraints.GreaterThan(1e-6)
    )
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()

    # optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 350
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    return model, likelihood


def utility(var, gamma):
    if gamma == 1:
        return torch.log(var)  # Log utility for gamma = 1
    else:
        return (var ** (1.0 - gamma)) / (1 - gamma)  # CRRA utility


def safe_utility(var, gamma):
    var = torch.clamp(var, min=1e-10)
    return utility(var, gamma)


def normalized_bond_holdings(xt, delta_plus, delta_minus, tau):
    delta_plus = torch.tensor(delta_plus,dtype=torch.float32,requires_grad=True)
    delta_minus = torch.tensor(delta_minus,dtype=torch.float32,requires_grad=True)

    delta = delta_plus - delta_minus
    transaction_costs = tau * torch.sum(delta_plus - delta_minus)
    # Compute bond holdings
    bt = 1.0 - torch.sum(xt + delta) - transaction_costs 
    # bt = torch.clamp(bt, min=0.0)
    return bt



def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    delta = delta_plus - delta_minus
    # Wealth at t+1
    pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
    # pi_t1 = torch.clamp(pi_t1, min=1e-10)  # Avoid division by zero or negative wealth
    # Portfolio weights at t+1
    xt1 = ((xt + delta) * Rt) / pi_t1
    # xt1 = torch.clamp(xt1, min=0.0, max=1.0)  # Ensure valid weights
    return pi_t1, xt1

def V_terminal(xT):
    return utility(1.0 - tau * torch.sum(torch.abs(xT)), gamma)

def bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf, convex_hull=None):
    # Compute bond holdings
    bt = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)

    # Simulate returns (expected returns for simplicity)
    Rt = torch.tensor(mu, dtype=torch.float32)

    # Compute next period wealth dynamics
    pi_t1, xt1 = normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf)

    # Determine whether the next state is inside or outside the NTR
    xt1_np = xt1.cpu().numpy()
    # Ensure xt1 requires gradient
    xt1.requires_grad_(True)

    if is_in_ntr(xt1_np, convex_hull):
        # Inside the NTR, use vt_next_in
        if isinstance(vt_next_in, gpytorch.models.ExactGP):
            vt_next_in.eval()
            vt_next_val = vt_next_in(xt1).mean  # Remove unsqueeze and use .mean
        elif callable(vt_next_in):
            vt_next_val = vt_next_in(xt1)
        elif vt_next_in is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_in to be a GP model or function.")

    else:
        # Outside the NTR, use vt_next_out
        if isinstance(vt_next_out, gpytorch.models.ExactGP):
            vt_next_out.eval()
            vt_next_val = vt_next_out(xt1).mean
        elif callable(vt_next_out):
            vt_next_val = vt_next_out(xt1)
        elif vt_next_out is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_out to be a GP model or function.")

    # Compute the value function
    # vt = safe_utility(pi_t1, gamma) + beta * torch.mean(pi_t1 ** (1.0 - gamma) * vt_next_val)
    vt = beta * torch.mean(pi_t1 ** (1.0 - gamma) * vt_next_val)

    return vt

def sample_state_points(D):
    small_value = 0.0
    points = []
    # Add corners of the simplex (ends)
    for i in range(2 ** D):
        # point = [(i >> j) & 1 for j in range(D)]
        point = [(1 - small_value) if ((i >> j) & 1) else small_value for j in range(D)]
        points.append(point)
    # points.append([0] * D)
    # Avoid adding the point [0, 0] by perturbing it
    points.append([small_value] * D)    
    # Add midpoints between all pairs of points
    for i in range(1, 2 ** D):
        for j in range(i):
            midpoint = [(a + b) / 2 for a, b in zip(points[i], points[j])]
            points.append(midpoint)
    # Add more midpoints by sampling regions with higher uncertainty (optional)
    points = [point for point in points if sum(point) <= 1]
    
    # Remove duplicates
    unique_points = []
    for point in points:
        if point not in unique_points:
            unique_points.append(point)
    
    return torch.tensor(unique_points, dtype=torch.float32)

# Sample state points from the simplex
def sample_state_points_simplex(D, N):
    # Generate a grid of points in the simplex
    def grid_simplex(n, k):
        # n: number of points, k: dimensions
        grid_points = []
        for _ in range(n):
            point = np.random.rand(k)
            point /= np.sum(point)
            grid_points.append(point)
        return np.array(grid_points)

    points = grid_simplex(N, D)
    # points = np.maximum(points, 0.)
    return torch.tensor(points, dtype=torch.float32)

def is_in_ntr(x, convex_hull):
    if convex_hull is None:
        return False

    new_point = np.array(x)
    hull = convex_hull
    A = hull.equations[:, :-1]
    b = -hull.equations[:, -1]
    inequalities = np.dot(A, new_point) + b
    return np.all(inequalities <= 1e-8)  # Allow for numerical tolerance

def approximate_ntr(vertices):
    # Compute convex hull of the vertices to represent the NTR
    if len(vertices) > 2:  # Convex hull requires at least 3 points
        vertices = torch.stack(vertices).detach().numpy()  # Convert to numpy
        hull = ConvexHull(vertices)  # Compute convex hull
        return vertices, hull
    else:
        # Return the vertices directly if fewer than 3 points are available
        return vertices, None

def MertonPoint(mu, Sigma, r, gamma):
    # Step 1: Compute the diagonal matrix Lambda with sqrt of diagonal elements of Sigma
    Lambda = np.diag(np.sqrt(np.diag(Sigma)))
    
    # Step 2: Compute (Lambda * Sigma * Lambda)^(-1)
    Lambda_Sigma_Lambda = np.dot(Lambda, np.dot(Sigma, Lambda))
    Lambda_Sigma_Lambda_inv = np.linalg.inv(Lambda_Sigma_Lambda)
    
    # Step 3: Compute mu - r
    mu_r = mu - r
    
    # Step 4: Compute the Merton portfolio weights
    pi = np.dot(Lambda_Sigma_Lambda_inv, mu_r / gamma)
    
    return pi


class PortfolioOptimization(Problem):
    def __init__(
        self,
        D,
        xt,
        vt_next_in,
        vt_next_out,
        t,
        T,
        beta,
        gamma,
        tau,
        Rf,
        mu,
        Sigma,
        include_consumption=False,
    ):
        self.D = D
        self.xt = xt
        self.vt_next_in = vt_next_in
        self.vt_next_out = vt_next_out
        self.t = t
        self.T = T
        self.beta = beta
        self.gamma = gamma
        self.tau = tau
        self.Rf = Rf
        self.mu = mu
        self.Sigma = Sigma
        self.include_consumption = include_consumption

        # Number of variables: c_t, delta_plus, delta_minus
        n = 2 * D  # Always include c_t

        # Number of constraints
        # m = 4*D + 3  # D constraints from xt + delta >= 0, and 3 scalar constraints
        m = D + 3  # D constraints from xt + delta >= 0, and 3 scalar constraints

        # Variable bounds
        lb = np.zeros(n)
        ub = np.ones(n)
        # if not self.include_consumption:
        #     # Fix c_t to zero
        #     lb[0] = 0.0
        #     ub[0] = 0.0  # Fix c_t to 0

        # Constraint bounds
        # cl = np.zeros(m)
        cl = np.zeros(m)
        cu = np.full(m, np.inf)  # All constraints are inequalities (>= 0)

        super().__init__(n=n, m=m, problem_obj=self, lb=lb, ub=ub, cl=cl, cu=cu)

    def objective(self, params):
        idx = 0
        delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf
            )
        
        if torch.isnan(vt).any() or torch.isinf(vt).any():
            raise ValueError("NaN or Inf detected in objective function!")

        # return -vt.item()
        # print(f"objective: {vt.item()}")
        return -vt

    def gradient(self, params):
        idx = 0
        delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf
        )

        vt.backward()

        grads = []
        grads.extend(delta_plus.grad.detach().numpy())
        grads.extend(delta_minus.grad.detach().numpy())
        grads_array = np.array(grads)
        if np.isnan(grads_array).any() or np.isinf(grads_array).any():
            print(f"gradient: {grads_array}")
            print(f"params: {params}")
            print(f"vt: {vt}")
            print(f"delta_plus: {delta_plus}")
            print(f"delta_minus: {delta_minus}")
            print(f"x_t: {self.xt}")
            print(f"vt_next_in: {self.vt_next_in}")
            print(f"vt_next_out: {self.vt_next_out}")   
            raise ValueError("NaN or Inf detected in gradients!")     

        # return -np.array(grads) #TODO
        return -np.array(grads)
    
    def compute_constraints(self, params_tensor):
        idx = 0
        delta_plus = params_tensor[idx : idx + self.D]
        delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
        delta = delta_plus - delta_minus

        # Compute constraints
        constraint_3 = self.xt + delta  # xt + delta >= 0
        total_sum = torch.sum(self.xt + delta)
        bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau)
        constraint_5 = 1.0 - (total_sum + bt)  # Total weights <= 1
        constraint_6 = total_sum + bt  # Total weights >= 0

        constraints_list = [
            constraint_3.view(-1),  # xt + delta constraints
            bt.view(-1),            # Bond holdings >= 0
            constraint_5.view(-1),  # Total weights <= 1
            constraint_6.view(-1),  # Total weights >= 0
        ]
        constraints_combined = torch.cat(constraints_list)
        return constraints_combined    

    def constraints(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)
        return constraints_combined.detach().numpy()

    def jacobian(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)

        jacobian_matrix = []

        # Compute gradients for each constraint
        for constraint in constraints_combined:
            grad = torch.autograd.grad(constraint, params_tensor, retain_graph=True)[0]
            jacobian_matrix.append(grad.detach().numpy())

        # Flatten the Jacobian matrix
        jacobian_array = np.vstack(jacobian_matrix)
        
        # Check for NaNs or Infs
        if np.isnan(jacobian_array).any() or np.isinf(jacobian_array).any():
            raise ValueError("NaN or Inf detected in Jacobian!")
        
        return jacobian_array.flatten()

def solve_bellman_with_ipopt(
    D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma,
    include_consumption=False, num_starts=7, drop_tolerance=0.2
):
    best_solution = None
    best_info = None
    best_obj_val = float('-inf')
    failed_attempts = 0
    max_failed_attempts = int(num_starts * drop_tolerance)

    def generate_feasible_initial_guess(X_t, D, tau, include_consumption=False):
        # Ensure that delta_plus and delta_minus satisfy the constraints
        delta_plus = np.zeros(D)
        delta_minus = np.zeros(D)

        total_available = 1.0 - np.sum(X_t)  # Total available room for adjustments

        for d in range(D):
            # Compute the available room for delta_plus and delta_minus based on X_t
            # Limit delta_plus to be within remaining available space and current asset holding
            max_delta_plus = min(X_t[d], total_available)

            # Randomly assign delta_plus within the available range
            delta_plus[d] = np.random.uniform(0, max_delta_plus)

            # Calculate the available room for delta_minus given delta_plus
            max_delta_minus = X_t[d] - delta_plus[d]
            delta_minus[d] = np.random.uniform(0, max_delta_minus)

            # Update the total available room for the next assets
            total_available -= delta_plus[d]

        # Compute transaction costs
        transaction_costs = tau * np.sum(delta_plus - delta_minus)

        # Compute bond holdings (bt), ensuring non-negative bond holdings
        bt = 1.0 - np.sum(X_t + delta_plus - delta_minus) - transaction_costs
        if bt < 0:
            raise ValueError("Initial guess led to infeasible bond holdings. This should not happen!")

        # Optionally include consumption
        c_t = 0.0 if not include_consumption else np.random.uniform(0, 0.05)

        return np.concatenate([ delta_plus, delta_minus])
    # Loop through multiple starting points (up to 5)
    for start_idx in range(num_starts):
        initial_guess = np.random.uniform(0, 0.9, size=1 + 2 * D)  # Random initial guess
        initial_guess = generate_feasible_initial_guess(xt.detach().numpy(), D, tau, include_consumption)
        # if not include_consumption:
        #     initial_guess[0] = 0.0  # Set c_t to zero if no consumption

        try:
            # Create the optimization problem
            print(f"time: {t} point: {xt}, start: {initial_guess}") 
            prob = PortfolioOptimization(
                D,
                xt,
                vt_next_in,
                vt_next_out,
                t,
                T,
                beta,
                gamma,
                tau,
                Rf,
                mu,
                Sigma,
                include_consumption=include_consumption,
            )


            prob.add_option("tol", 1e-6)
            prob.add_option("max_iter", 300)
            prob.add_option("acceptable_tol", 1e-5)
            prob.add_option("print_level", 4)
            # prob.add_option("gradient_approximation", "exact")  # More accurate but slower
            # prob.add_option("gradient_approximation", "finite-difference-values")  # More accurate but slower

            prob.add_option("derivative_test", "second-order")
            prob.add_option("derivative_test_tol", 1e-6)
            prob.add_option("mu_strategy", "adaptive")  # Adaptive step size strategy
            prob.add_option("mu_oracle", "quality-function")  # Control step quality
            prob.add_option("honor_original_bounds", "yes")
            prob.add_option("barrier_tol_factor", 1e-5)

            solution, info = prob.solve(initial_guess)

            # Check if this solution is better than the current best
            if info['status'] == 0 and (best_solution is None or info['obj_val'] > best_obj_val):
                best_solution = solution
                best_info = info
                best_obj_val = info['obj_val']

        except Exception as e:
            print(f"Optimization failed for start {start_idx}: {e}")
            failed_attempts += 1
            # If too many failures occur, drop this point
            if failed_attempts > max_failed_attempts:
                print(f"Exceeded maximum allowed failed attempts: {max_failed_attempts}")
                return None, None
            continue

    if best_solution is None:
        bt = normalized_bond_holdings(xt, torch.zeros(D), torch.zeros(D), tau).item()
        print(f"Xt: {xt}")
        print(f"vt_next_in: {vt_next_in}") 
        print(f"vt_next_out: {vt_next_out}")
        print(f"t: {t}")
        print(f"T: {T}")
        print(f"bt: {bt}")
        raise ValueError("All initial guesses led to infeasibility.")

    # After finding the best solution, extract the variables
    idx = 0
    delta_plus_opt = best_solution[idx : idx + D]
    delta_minus_opt = best_solution[idx + D : idx + 2 * D]
    delta_opt = delta_plus_opt - delta_minus_opt

    # Compute omega_i_t and bond holdings (bt)
    omega_i_t = xt.cpu().numpy() + delta_opt
    print(f"delta_plus_opt: {delta_plus_opt}, delta_minus_opt: {delta_minus_opt}, delta_opt: {delta_opt}, omega_i_t: {omega_i_t}")
    return delta_plus_opt, delta_minus_opt, delta_opt, omega_i_t


def approximate_ntr(vt_next_in, vt_next_out, D, t, T, beta, gamma, tau, Rf, mu, Sigma):
    # Step 1: Sample state points
    tilde_X_t = sample_state_points(D)
    N = len(tilde_X_t)
    tilde_omega_t = []

    for i in range(N):
        tilde_x_i_t = tilde_X_t[i]
        # Step 2: Solve optimization problem
        # delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
        #     D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        # )
        delta_plus, delta_minus, delta, omega_i_t = solve_bellman_with_ipopt(
            D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        )
        # Step 3: Compute NTR vertices
        tilde_omega_i_t = (tilde_x_i_t + delta).cpu().numpy()
        tilde_omega_t.append(tilde_omega_i_t)

    # Step 4: Compute convex hull of the vertices to represent the NTR
    tilde_omega_t = np.array(tilde_omega_t)
    if len(tilde_omega_t) >= D + 1:
        convex_hull = ConvexHull(tilde_omega_t)
    else:
        convex_hull = None  # Cannot compute convex hull with fewer points

    return tilde_omega_t, convex_hull

def dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma):
    # Initialize value function V
    V = [[None, None] for _ in range(T + 1)]
    
    # Set terminal value function
    V[T][0] = V_terminal  # For inside NTR
    V[T][1] = V_terminal  # For outside NTR

    NTRs = [None for _ in range(T)]  # Store NTRs for each period

    for t in reversed(range(T)):
        print(f"Time step {t}")

        # Step 2a: Approximate NTR
        tilde_omega_t, convex_hull = approximate_ntr(V[t + 1][0], V[t + 1][1], D, t, T, beta, gamma, tau, Rf, mu, Sigma)
        NTRs[t] = convex_hull

        # Step 2b: Sample state points
        X_t = sample_state_points_simplex(D, N)
        data_in = []
        data_out = []

        for i in range(len(X_t)):
            x_i_t = X_t[i]
            # Step 2c: Solve optimization problem
            # delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
            #     D, x_i_t, V[t + 1][0], V[t + 1][1], t + 1, T, beta, gamma, tau, Rf, mu, Sigma
            # )
            delta_plus, delta_minus, delta, omega_i_t = solve_bellman_with_ipopt(
                D, x_i_t, V[t + 1][0], V[t + 1][1], t, T, beta, gamma, tau, Rf, mu, Sigma
            )
            
            # Compute value using Bellman equation, selecting the correct V[t+1]
            v_i_t = bellman_equation(V[t + 1][0], V[t + 1][1], x_i_t, delta_plus, delta_minus, beta, gamma, tau, Rf)

            # Determine if the point is inside the NTR and append to the respective data set
            x_i_t_np = x_i_t.detach().numpy()
            in_ntr = is_in_ntr(x_i_t_np, convex_hull)
            if in_ntr:
                data_in.append((x_i_t_np, v_i_t.detach().numpy()))  # Ensure numpy values
            else:
                data_out.append((x_i_t_np, v_i_t.detach().numpy()))  # Ensure numpy values
            print(x_i_t , delta_plus, delta_minus, delta, omega_i_t)

        # # Step 2e: Train GPR models for inside and outside NTR
        if data_in:
            # Print for debugging
            print("Data out structure:", [(d[0], d[1].shape if isinstance(d[1], torch.Tensor) else "scalar") for d in data_in])
            
            try:
                # Convert list of data_out values to tensors
                train_x_in = torch.tensor([d[0] for d in data_in], dtype=torch.float32)
                
                # Handle scalar values by converting them into 1D tensors with unsqueeze
                train_y_in = torch.tensor([d[1].item() if isinstance(d[1], torch.Tensor) else d[1] for d in data_in], dtype=torch.float32).unsqueeze(1)

                # Ensure that train_y_out is the correct shape
                model_in, likelihood_in = train_gp_model(train_x_in,train_y_in)

                if model_in is not None:
                    V[t][0] = model_in  # Surrogate for outside NTR
            except Exception as e:
                print("Error in constructing train_y_in:", e)
        else:
            V[t][1] = V_terminal  # Fallback if no outside NTR data     

        # Step 2e: Train GPR models for inside and outside NTR
        if data_out:
            try:
                # Print for debugging
                print("Data out structure:", [(d[0], d[1].shape if isinstance(d[1], torch.Tensor) else "scalar") for d in data_out])

                # Convert list of data_out values to tensors
                train_x_out = torch.tensor([d[0] for d in data_out], dtype=torch.float32)

                # Handle scalar values and wrap them into 1D tensors with unsqueeze
                train_y_out = torch.tensor([d[1] for d in data_out], dtype=torch.float32).unsqueeze(1)

                # Train GP model
                model_out, likelihood_out = train_gp_model(train_x_out, train_y_out)

                if model_out is not None:
                    V[t][1] = model_out  # Surrogate for outside NTR
                else:
                    V[t][1] = V_terminal
            except Exception as e:
                print("Error in constructing train_y_out:", e)
                print(f"type(data_out): {type(data_out)}")
        else:
            V[t][1] = V_terminal  # Fallback if no outside NTR data
        
    return V, NTRs

# Parameters
T = 6  # Time horizon
N = 100  # Number of sample points
D = 2  # Number of risky assets

# Run the dynamic programming algorithm
V, NTRs = dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma)

In [ ]:
import numpy as np

import torch
import torch.autograd as autograd

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

import cyipopt
from cyipopt import Problem
from scipy.spatial import ConvexHull
from scipy.optimize import minimize
from numpy.polynomial.hermite import hermgauss

import logging

# Set up logging configuration
logging.basicConfig(filename='optimization_log.txt', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')


# Set random seed for reproducibility
np.random.seed(2001)
torch.manual_seed(2001)

# Parameters
T = 10  # Time horizon
D = 2  # Number of risky assets
r = 0.02  # Risk-free return in pct.
Rf = np.exp(r)  # Risk-free return
Rf = r  # Risk-free return
tau = 0.005  # Transaction cost rate
# tau = 0.0
beta = 0.975  # Discount factor
gamma = 3.0  # Risk aversion coefficient

# Risky assets - deterministic
mu = np.array([0.07, 0.07])
Sigma = np.array([[0.2, 0], [0, 0.2]])
Lambda = np.diag(np.sqrt(np.diag(Sigma)))

# Include consumption flag
include_consumption = False  # Set to True to include consumption


# Define the GPR model with ARD
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def train_gp_model(train_x, train_y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood(
        noise_constraint=gpytorch.constraints.GreaterThan(1e-6)
    )
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()

    # optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 200
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    return model, likelihood


def utility(var, gamma):
    if gamma == 1:
        return torch.log(var)  # Log utility for gamma = 1
    else:
        return (var ** (1.0 - gamma)) / (1 - gamma)  # CRRA utility


def safe_utility(var, gamma):
    var = torch.clamp(var, min=1e-10)
    return utility(var, gamma)


def normalized_bond_holdings(xt, delta_plus, delta_minus, tau):
    delta_plus = torch.tensor(delta_plus,dtype=torch.float32,requires_grad=True)
    delta_minus = torch.tensor(delta_minus,dtype=torch.float32,requires_grad=True)

    delta = delta_plus - delta_minus
    transaction_costs = tau * torch.sum(delta_plus - delta_minus)
    # Compute bond holdings
    bt = 1.0 - torch.sum(xt + delta) - transaction_costs 
    # bt = torch.clamp(bt, min=0.0)
    return bt



def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    delta = delta_plus - delta_minus
    # Wealth at t+1
    pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
    # pi_t1 = torch.clamp(pi_t1, min=1e-10)  # Avoid division by zero or negative wealth
    # Portfolio weights at t+1
    xt1 = ((xt + delta) * Rt) / pi_t1
    # xt1 = torch.clamp(xt1, min=0.0, max=1.0)  # Ensure valid weights
    return pi_t1, xt1

def V_terminal(xT):
    return utility(1.0 - tau * torch.sum(torch.abs(xT)), gamma)

def bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf, convex_hull=None):
    # Compute bond holdings
    bt = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)

    # Simulate returns (expected returns for simplicity)
    Rt = torch.tensor(mu, dtype=torch.float32)

    # Compute next period wealth dynamics
    pi_t1, xt1 = normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf)

    # Determine whether the next state is inside or outside the NTR
    xt1_np = xt1.cpu().numpy()
    # Ensure xt1 requires gradient
    xt1.requires_grad_(True)

    if is_in_ntr(xt1_np, convex_hull):
        # Inside the NTR, use vt_next_in
        if isinstance(vt_next_in, gpytorch.models.ExactGP):
            vt_next_in.eval()
            vt_next_val = vt_next_in(xt1).mean  # Remove unsqueeze and use .mean
        elif callable(vt_next_in):
            vt_next_val = vt_next_in(xt1)
        elif vt_next_in is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_in to be a GP model or function.")

    else:
        # Outside the NTR, use vt_next_out
        if isinstance(vt_next_out, gpytorch.models.ExactGP):
            vt_next_out.eval()
            vt_next_val = vt_next_out(xt1).mean
        elif callable(vt_next_out):
            vt_next_val = vt_next_out(xt1)
        elif vt_next_out is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_out to be a GP model or function.")

    # Compute the value function
    # vt = safe_utility(pi_t1, gamma) + beta * torch.mean(pi_t1 ** (1.0 - gamma) * vt_next_val)
    vt = beta * torch.mean(pi_t1 ** (1.0 - gamma) * vt_next_val)

    return vt

def sample_state_points(D):
    small_value = 0.0
    points = []
    # Add corners of the simplex (ends)
    for i in range(2 ** D):
        # point = [(i >> j) & 1 for j in range(D)]
        point = [(1 - small_value) if ((i >> j) & 1) else small_value for j in range(D)]
        points.append(point)
    # points.append([0] * D)
    # Avoid adding the point [0, 0] by perturbing it
    points.append([small_value] * D)    
    # Add midpoints between all pairs of points
    for i in range(1, 2 ** D):
        for j in range(i):
            midpoint = [(a + b) / 2 for a, b in zip(points[i], points[j])]
            points.append(midpoint)
    # Add more midpoints by sampling regions with higher uncertainty (optional)
    points = [point for point in points if sum(point) <= 1]
    
    # Remove duplicates
    unique_points = []
    for point in points:
        if point not in unique_points:
            unique_points.append(point)
    
    return torch.tensor(unique_points, dtype=torch.float32)

# Sample state points from the simplex
def sample_state_points_simplex(D, N):
    # Generate a grid of points in the simplex
    def grid_simplex(n, k):
        # n: number of points, k: dimensions
        grid_points = []
        for _ in range(n):
            point = np.random.rand(k)
            point /= np.sum(point)
            grid_points.append(point)
        return np.array(grid_points)

    points = grid_simplex(N, D)
    # points = np.maximum(points, 0.)
    return torch.tensor(points, dtype=torch.float32)

def is_in_ntr(x, convex_hull):
    if convex_hull is None:
        return False

    new_point = np.array(x)
    hull = convex_hull
    A = hull.equations[:, :-1]
    b = -hull.equations[:, -1]
    inequalities = np.dot(A, new_point) + b
    return np.all(inequalities <= 1e-8)  # Allow for numerical tolerance

def approximate_ntr(vertices):
    # Compute convex hull of the vertices to represent the NTR
    if len(vertices) > 2:  # Convex hull requires at least 3 points
        vertices = torch.stack(vertices).cpu().numpy()  # Convert to numpy
        hull = ConvexHull(vertices)  # Compute convex hull
        return vertices, hull
    else:
        # Return the vertices directly if fewer than 3 points are available
        return vertices, None

def MertonPoint(mu, Sigma, r, gamma):
    # Step 1: Compute the diagonal matrix Lambda with sqrt of diagonal elements of Sigma
    Lambda = np.diag(np.sqrt(np.diag(Sigma)))
    
    # Step 2: Compute (Lambda * Sigma * Lambda)^(-1)
    Lambda_Sigma_Lambda = np.dot(Lambda, np.dot(Sigma, Lambda))
    Lambda_Sigma_Lambda_inv = np.linalg.inv(Lambda_Sigma_Lambda)
    
    # Step 3: Compute mu - r
    mu_r = mu - r
    
    # Step 4: Compute the Merton portfolio weights
    pi = np.dot(Lambda_Sigma_Lambda_inv, mu_r / gamma)
    
    return pi


class PortfolioOptimization(Problem):
    def __init__(
        self,
        D,
        xt,
        vt_next_in,
        vt_next_out,
        t,
        T,
        beta,
        gamma,
        tau,
        Rf,
        mu,
        Sigma,
        include_consumption=False,
    ):
        self.D = D
        self.xt = xt
        self.vt_next_in = vt_next_in
        self.vt_next_out = vt_next_out
        self.t = t
        self.T = T
        self.beta = beta
        self.gamma = gamma
        self.tau = tau
        self.Rf = Rf
        self.mu = mu
        self.Sigma = Sigma
        self.include_consumption = include_consumption

        # Number of variables: c_t, delta_plus, delta_minus
        n = 2 * D  # Always include c_t

        # Number of constraints
        # m = 4*D + 3  # D constraints from xt + delta >= 0, and 3 scalar constraints
        m = D + 3  # D constraints from xt + delta >= 0, and 3 scalar constraints

        # Variable bounds
        lb = np.zeros(n)
        ub = np.ones(n)
        # if not self.include_consumption:
        #     # Fix c_t to zero
        #     lb[0] = 0.0
        #     ub[0] = 0.0  # Fix c_t to 0

        # Constraint bounds
        # cl = np.zeros(m)
        cl = np.zeros(m)
        cu = np.full(m, np.inf)  # All constraints are inequalities (>= 0)

        super().__init__(n=n, m=m, problem_obj=self, lb=lb, ub=ub, cl=cl, cu=cu)

    def objective(self, params):
        idx = 0
        delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf
            )
        
        if torch.isnan(vt).any() or torch.isinf(vt).any():
            raise ValueError("NaN or Inf detected in objective function!")

        # return -vt.item()
        # print(f"objective: {vt.item()}")
        return -vt

    def gradient(self, params):
        idx = 0
        delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf
        )

        vt.backward()

        grads = []
        grads.extend(delta_plus.grad.cpu().numpy())
        grads.extend(delta_minus.grad.cpu().numpy())
        grads_array = np.array(grads)
        if np.isnan(grads_array).any() or np.isinf(grads_array).any():
            print(f"gradient: {grads_array}")
            print(f"params: {params}")
            print(f"vt: {vt}")
            print(f"delta_plus: {delta_plus}")
            print(f"delta_minus: {delta_minus}")
            print(f"x_t: {self.xt}")
            print(f"vt_next_in: {self.vt_next_in}")
            print(f"vt_next_out: {self.vt_next_out}")   
            raise ValueError("NaN or Inf detected in gradients!")     

        # return -np.array(grads) #TODO
        return -np.array(grads)
    
    def compute_constraints(self, params_tensor):
        idx = 0
        delta_plus = params_tensor[idx : idx + self.D]
        delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
        delta = delta_plus - delta_minus

        # Compute constraints
        constraint_3 = self.xt + delta  # xt + delta >= 0
        total_sum = torch.sum(self.xt + delta)
        bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau)
        constraint_5 = 1.0 - (total_sum + bt)  # Total weights <= 1
        constraint_6 = total_sum + bt  # Total weights >= 0

        constraints_list = [
            constraint_3.view(-1),  # xt + delta constraints
            bt.view(-1),            # Bond holdings >= 0
            constraint_5.view(-1),  # Total weights <= 1
            constraint_6.view(-1),  # Total weights >= 0
        ]
        constraints_combined = torch.cat(constraints_list)
        return constraints_combined    

    def constraints(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)
        return constraints_combined.cpu().numpy()

    def jacobian(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)

        jacobian_matrix = []

        # Compute gradients for each constraint
        for constraint in constraints_combined:
            grad = torch.autograd.grad(constraint, params_tensor, retain_graph=True)[0]
            jacobian_matrix.append(grad.cpu().numpy())

        # Flatten the Jacobian matrix
        jacobian_array = np.vstack(jacobian_matrix)
        
        # Check for NaNs or Infs
        if np.isnan(jacobian_array).any() or np.isinf(jacobian_array).any():
            raise ValueError("NaN or Inf detected in Jacobian!")
        
        return jacobian_array.flatten()

def solve_bellman_with_ipopt(
    D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma,
    include_consumption=False, num_starts=7, drop_tolerance=0.2
):
    best_solution = None
    best_info = None
    best_obj_val = float('-inf')
    failed_attempts = 0
    max_failed_attempts = int(num_starts * drop_tolerance)

    def generate_feasible_initial_guess(X_t, D, tau, include_consumption=False):
        # Ensure that delta_plus and delta_minus satisfy the constraints
        delta_plus = np.zeros(D)
        delta_minus = np.zeros(D)

        total_available = 1.0 - np.sum(X_t)  # Total available room for adjustments

        for d in range(D):
            # Compute the available room for delta_plus and delta_minus based on X_t
            # Limit delta_plus to be within remaining available space and current asset holding
            max_delta_plus = min(X_t[d], total_available)

            # Randomly assign delta_plus within the available range
            delta_plus[d] = np.random.uniform(0, max_delta_plus)

            # Calculate the available room for delta_minus given delta_plus
            max_delta_minus = X_t[d] - delta_plus[d]
            delta_minus[d] = np.random.uniform(0, max_delta_minus)

            # Update the total available room for the next assets
            total_available -= delta_plus[d]

        # Compute transaction costs
        transaction_costs = tau * np.sum(delta_plus - delta_minus)

        # Compute bond holdings (bt), ensuring non-negative bond holdings
        bt = 1.0 - np.sum(X_t + delta_plus - delta_minus) - transaction_costs
        if bt < 0:
            raise ValueError("Initial guess led to infeasible bond holdings. This should not happen!")

        # Optionally include consumption
        c_t = 0.0 if not include_consumption else np.random.uniform(0, 0.05)

        return np.concatenate([ delta_plus, delta_minus])
    # Loop through multiple starting points (up to 5)
    for start_idx in range(num_starts):
        initial_guess = np.random.uniform(0, 0.9, size=1 + 2 * D)  # Random initial guess
        initial_guess = generate_feasible_initial_guess(xt.cpu().numpy(), D, tau, include_consumption)
        # if not include_consumption:
        #     initial_guess[0] = 0.0  # Set c_t to zero if no consumption

        try:
            # Create the optimization problem
            print(f"time: {t} point: {xt}, start: {initial_guess}") 
            prob = PortfolioOptimization(
                D,
                xt,
                vt_next_in,
                vt_next_out,
                t,
                T,
                beta,
                gamma,
                tau,
                Rf,
                mu,
                Sigma,
                include_consumption=include_consumption,
            )


            prob.add_option("tol", 1e-6)
            prob.add_option("max_iter", 300)
            prob.add_option("acceptable_tol", 1e-5)
            prob.add_option("print_level", 4)
            # prob.add_option("gradient_approximation", "exact")  # More accurate but slower
            # prob.add_option("gradient_approximation", "finite-difference-values")  # More accurate but slower

            prob.add_option("derivative_test", "second-order")
            prob.add_option("derivative_test_tol", 1e-6)
            prob.add_option("mu_strategy", "adaptive")  # Adaptive step size strategy
            prob.add_option("mu_oracle", "quality-function")  # Control step quality
            prob.add_option("honor_original_bounds", "yes")
            prob.add_option("barrier_tol_factor", 1e-5)

            solution, info = prob.solve(initial_guess)

            # Check if this solution is better than the current best
            if info['status'] == 0 and (best_solution is None or info['obj_val'] > best_obj_val):
                best_solution = solution
                best_info = info
                best_obj_val = info['obj_val']

        except Exception as e:
            print(f"Optimization failed for start {start_idx}: {e}")
            failed_attempts += 1
            # If too many failures occur, drop this point
            if failed_attempts > max_failed_attempts:
                print(f"Exceeded maximum allowed failed attempts: {max_failed_attempts}")
                return None, None
            continue

    if best_solution is None:
        bt = normalized_bond_holdings(xt, torch.zeros(D), torch.zeros(D), tau).item()
        print(f"Xt: {xt}")
        print(f"vt_next_in: {vt_next_in}") 
        print(f"vt_next_out: {vt_next_out}")
        print(f"t: {t}")
        print(f"T: {T}")
        print(f"bt: {bt}")
        raise ValueError("All initial guesses led to infeasibility.")

    # After finding the best solution, extract the variables
    idx = 0
    delta_plus_opt = best_solution[idx : idx + D]
    delta_minus_opt = best_solution[idx + D : idx + 2 * D]
    delta_opt = delta_plus_opt - delta_minus_opt

    # Compute omega_i_t and bond holdings (bt)
    omega_i_t = xt.cpu().numpy() + delta_opt
    bt = normalized_bond_holdings(
        xt, torch.tensor(delta_plus_opt), torch.tensor(delta_minus_opt), tau
    ).item()

    return delta_plus_opt, delta_minus_opt, delta_opt, omega_i_t, bt


def approximate_ntr(vt_next_in, vt_next_out, D, t, T, beta, gamma, tau, Rf, mu, Sigma):
    # Step 1: Sample state points
    tilde_X_t = sample_state_points(D)
    N = len(tilde_X_t)
    tilde_omega_t = []

    for i in range(N):
        tilde_x_i_t = tilde_X_t[i]
        # Step 2: Solve optimization problem
        # delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
        #     D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        # )
        delta_plus, delta_minus, delta, omega_i_t, b_t = solve_bellman_with_ipopt(
            D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        )
        # Step 3: Compute NTR vertices
        tilde_omega_i_t = (tilde_x_i_t + delta).cpu().numpy()
        tilde_omega_t.append(tilde_omega_i_t)

    # Step 4: Compute convex hull of the vertices to represent the NTR
    tilde_omega_t = np.array(tilde_omega_t)
    if len(tilde_omega_t) >= D + 1:
        convex_hull = ConvexHull(tilde_omega_t)
    else:
        convex_hull = None  # Cannot compute convex hull with fewer points

    return tilde_omega_t, convex_hull

def dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma):
    # Initialize value function V
    V = [[None, None] for _ in range(T + 1)]
    
    # Set terminal value function
    V[T][0] = V_terminal  # For inside NTR
    V[T][1] = V_terminal  # For outside NTR

    NTRs = [None for _ in range(T)]  # Store NTRs for each period

    for t in reversed(range(T)):
        print(f"Time step {t}")

        # Step 2a: Approximate NTR
        tilde_omega_t, convex_hull = approximate_ntr(V[t + 1][0], V[t + 1][1], D, t, T, beta, gamma, tau, Rf, mu, Sigma)
        NTRs[t] = convex_hull

        # Step 2b: Sample state points
        X_t = sample_state_points_simplex(D, N)
        data_in = []
        data_out = []

        for i in range(len(X_t)):
            x_i_t = X_t[i]
            # Step 2c: Solve optimization problem
            # delta_plus, delta_minus, delta, omega_i_t, b_t = solve_optimization(
            #     D, x_i_t, V[t + 1][0], V[t + 1][1], t + 1, T, beta, gamma, tau, Rf, mu, Sigma
            # )
            delta_plus, delta_minus, delta, omega_i_t, b_t = solve_bellman_with_ipopt(
                D, x_i_t, V[t + 1][0], V[t + 1][1], t, T, beta, gamma, tau, Rf, mu, Sigma
            )
            
            # Compute value using Bellman equation, selecting the correct V[t+1]
            v_i_t = bellman_equation(V[t + 1][0], V[t + 1][1], x_i_t, delta_plus, delta_minus, beta, gamma, tau, Rf)

            # Determine if the point is inside the NTR and append to the respective data set
            x_i_t_np = x_i_t.cpu().numpy()
            in_ntr = is_in_ntr(x_i_t_np, convex_hull)
            if in_ntr:
                data_in.append((x_i_t_np, v_i_t.cpu().numpy()))  # Ensure numpy values
            else:
                data_out.append((x_i_t_np, v_i_t.cpu().numpy()))  # Ensure numpy values
            print(x_i_t , delta_plus, delta_minus, delta, omega_i_t)

        # # Step 2e: Train GPR models for inside and outside NTR
        if data_in:
            # Print for debugging
            print("Data out structure:", [(d[0], d[1].shape if isinstance(d[1], torch.Tensor) else "scalar") for d in data_in])
            
            try:
                # Convert list of data_out values to tensors
                train_x_in = torch.tensor([d[0] for d in data_in], dtype=torch.float32)
                
                # Handle scalar values by converting them into 1D tensors with unsqueeze
                train_y_in = torch.tensor([d[1].item() if isinstance(d[1], torch.Tensor) else d[1] for d in data_in], dtype=torch.float32).unsqueeze(1)

                # Ensure that train_y_out is the correct shape
                model_in, likelihood_in = train_gp_model(train_x_in,train_y_in)

                if model_in is not None:
                    V[t][0] = model_in  # Surrogate for outside NTR
            except Exception as e:
                print("Error in constructing train_y_in:", e)
        else:
            V[t][1] = V_terminal  # Fallback if no outside NTR data     

        # Step 2e: Train GPR models for inside and outside NTR
        if data_out:
            try:
                # Print for debugging
                print("Data out structure:", [(d[0], d[1].shape if isinstance(d[1], torch.Tensor) else "scalar") for d in data_out])

                # Convert list of data_out values to tensors
                train_x_out = torch.tensor([d[0] for d in data_out], dtype=torch.float32)

                # Handle scalar values and wrap them into 1D tensors with unsqueeze
                train_y_out = torch.tensor([d[1] for d in data_out], dtype=torch.float32).unsqueeze(1)

                # Train GP model
                model_out, likelihood_out = train_gp_model(train_x_out, train_y_out)

                if model_out is not None:
                    V[t][1] = model_out  # Surrogate for outside NTR
                else:
                    V[t][1] = V_terminal
            except Exception as e:
                print("Error in constructing train_y_out:", e)
                print(f"type(data_out): {type(data_out)}")
        else:
            V[t][1] = V_terminal  # Fallback if no outside NTR data
        
    return V, NTRs

# Parameters
T = 6  # Time horizon
N = 100  # Number of sample points
D = 2  # Number of risky assets

# Run the dynamic programming algorithm
V, NTRs = dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma)

# O1Preview. 

In [4]:
import numpy as np
import torch
import torch.autograd as autograd

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

import cyipopt
from cyipopt import Problem
from scipy.spatial import ConvexHull

import logging

# Set up logging configuration
logging.basicConfig(filename='optimization_log.txt', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Set random seed for reproducibility
np.random.seed(2001)
torch.manual_seed(2001)

# Parameters
T = 10  # Time horizon
D = 2  # Number of risky assets
r = 0.02  # Risk-free return in pct.
Rf = np.exp(r)  # Risk-free return
Rf = r  # Risk-free return
tau = 0.005  # Transaction cost rate
beta = 0.975  # Discount factor
gamma = 3.0  # Risk aversion coefficient

# Risky assets - deterministic
mu = np.array([0.07, 0.07])
Sigma = np.array([[0.2, 0], [0, 0.2]])

# Include consumption flag
include_consumption = False  # Set to True to include consumption

# Define the GPR model with ARD
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_gp_model(train_x, train_y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood(
        noise_constraint=gpytorch.constraints.GreaterThan(1e-6)
    )
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 350
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    return model, likelihood

def utility(var, gamma):
    if gamma == 1:
        return torch.log(var)  # Log utility for gamma = 1
    else:
        return (var ** (1.0 - gamma)) / (1 - gamma)  # CRRA utility

def safe_utility(var, gamma):
    var = torch.clamp(var, min=1e-10)
    return utility(var, gamma)

def normalized_bond_holdings(xt, delta_plus, delta_minus, tau):
    delta = delta_plus - delta_minus
    transaction_costs = tau * torch.sum(delta_plus - delta_minus)
    # Compute bond holdings
    bt = 1.0 - torch.sum(xt + delta) - transaction_costs
    return bt

def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    delta = delta_plus - delta_minus
    # Wealth at t+1
    pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
    # Portfolio weights at t+1
    xt1 = ((xt + delta) * Rt) / pi_t1
    return pi_t1, xt1

def V_terminal(xT):
    return utility(1.0 - tau * torch.sum(torch.abs(xT)), gamma)

def bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf, convex_hull=None):
    # Compute bond holdings
    bt = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)

    # Simulate returns (expected returns for simplicity)
    Rt = torch.tensor(mu, dtype=torch.float32)

    # Compute next period wealth dynamics
    pi_t1, xt1 = normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf)

    # Do not set requires_grad on xt1

    # Determine whether the next state is inside or outside the NTR
    xt1_np = xt1.detach().cpu().numpy()

    if is_in_ntr(xt1_np, convex_hull):
        # Inside the NTR, use vt_next_in
        if isinstance(vt_next_in, gpytorch.models.ExactGP):
            vt_next_in.eval()
            vt_next_val = vt_next_in(xt1).mean  # Predictive mean with gradients
        elif callable(vt_next_in):
            vt_next_val = vt_next_in(xt1)
        elif vt_next_in is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_in to be a GP model or function.")
    else:
        # Outside the NTR, use vt_next_out
        if isinstance(vt_next_out, gpytorch.models.ExactGP):
            vt_next_out.eval()
            vt_next_val = vt_next_out(xt1).mean
        elif callable(vt_next_out):
            vt_next_val = vt_next_out(xt1)
        elif vt_next_out is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_out to be a GP model or function.")

    # Compute the value function
    vt = beta * (pi_t1 ** (1.0 - gamma)) * vt_next_val

    return vt

def sample_state_points(D):
    small_value = 0.0
    points = []
    # Add corners of the simplex (ends)
    for i in range(2 ** D):
        point = [(1 - small_value) if ((i >> j) & 1) else small_value for j in range(D)]
        points.append(point)
    # Add the point [small_value, ..., small_value]
    points.append([small_value] * D)
    # Add midpoints between all pairs of points
    for i in range(1, 2 ** D):
        for j in range(i):
            midpoint = [(a + b) / 2 for a, b in zip(points[i], points[j])]
            points.append(midpoint)
    # Ensure points are within the simplex
    points = [point for point in points if sum(point) <= 1]
    # Remove duplicates
    unique_points = []
    for point in points:
        if point not in unique_points:
            unique_points.append(point)
    return torch.tensor(unique_points, dtype=torch.float32)

def sample_state_points_simplex(D, N):
    # Generate random points in the simplex
    def random_points_in_simplex(n, k):
        points = np.random.dirichlet(np.ones(k), size=n)
        return points
    points = random_points_in_simplex(N, D)
    return torch.tensor(points, dtype=torch.float32)

def is_in_ntr(x, convex_hull):
    if convex_hull is None:
        return False
    new_point = np.array(x)
    hull = convex_hull
    A = hull.equations[:, :-1]
    b = -hull.equations[:, -1]
    inequalities = np.dot(A, new_point) + b
    return np.all(inequalities <= 1e-5)  # Allow for numerical tolerance

def MertonPoint(mu, Sigma, r, gamma):
    # Compute the Merton portfolio weights
    Lambda = np.diag(np.sqrt(np.diag(Sigma)))
    Lambda_Sigma_Lambda = np.dot(Lambda, np.dot(Sigma, Lambda))
    Lambda_Sigma_Lambda_inv = np.linalg.inv(Lambda_Sigma_Lambda)
    mu_r = mu - r
    pi = np.dot(Lambda_Sigma_Lambda_inv, mu_r / gamma)
    return pi

class PortfolioOptimization(cyipopt.Problem):
    def __init__(
        self,
        D,
        xt,
        vt_next_in,
        vt_next_out,
        t,
        T,
        beta,
        gamma,
        tau,
        Rf,
        mu,
        Sigma,
        convex_hull=None,
        include_consumption=False,
    ):
        self.D = D
        self.xt = xt.detach().clone()  # Ensure self.xt is a leaf variable
        self.vt_next_in = vt_next_in
        self.vt_next_out = vt_next_out
        self.t = t
        self.T = T
        self.beta = beta
        self.gamma = gamma
        self.tau = tau
        self.Rf = Rf
        self.mu = mu
        self.Sigma = Sigma
        self.convex_hull = convex_hull
        self.include_consumption = include_consumption

        # Number of variables: delta_plus, delta_minus
        self.n = 2 * D

        # Number of constraints: D constraints from xt + delta >= 0, and 3 scalar constraints
        self.m = D + 3

        # Variable bounds
        lb = np.zeros(self.n)
        ub = np.ones(self.n)

        # Constraint bounds
        cl = np.zeros(self.m)
        cu = np.full(self.m, np.inf)  # All constraints are inequalities (>= 0)

        super().__init__(n=self.n, m=self.m, problem_obj=self, lb=lb, ub=ub, cl=cl, cu=cu)

    def objective(self, params):
        idx = 0

        # Convert params to a tensor with gradient tracking
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        delta_plus = params_tensor[idx : idx+self.D]
        delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
        
        # Compute the value function
        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf,
            self.convex_hull
        )
        
        if torch.isnan(vt).any() or torch.isinf(vt).any():
            raise ValueError("NaN or Inf detected in objective function!")
        
        # Logging for debugging
        logging.info(f"delta_plus: {delta_plus.detach().cpu().numpy()}, delta_minus: {delta_minus.detach().cpu().numpy()}")
        logging.info(f"Objective Value (vt): {vt.item()}")
        
        return vt.item()  # IPOPT minimizes, so negate to maximize

    # def gradient(self, params):
    #     idx = 0
    #     delta_plus = torch.tensor(params[idx : idx + self.D], dtype=torch.float32, requires_grad=True)
    #     delta_minus = torch.tensor(params[idx + self.D : idx + 2 * self.D], dtype=torch.float32, requires_grad=True)

    #     xt = self.xt  # Do not set requires_grad on xt

    #     vt = bellman_equation(
    #         self.vt_next_in,
    #         self.vt_next_out,
    #         xt,
    #         delta_plus,
    #         delta_minus,
    #         self.beta,
    #         self.gamma,
    #         self.tau,
    #         self.Rf,
    #         self.convex_hull
    #     )

    #     vt.backward()

    #     grads = []
    #     grads.extend(delta_plus.grad.detach().cpu().numpy())
    #     grads.extend(delta_minus.grad.detach().cpu().numpy())

    #     return -np.array(grads)
    def gradient(self, params):
        # Convert params to a tensor with gradient tracking
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        delta_plus = params_tensor[:self.D]
        delta_minus = params_tensor[self.D:2 * self.D]
        
        # Compute the value function
        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf,
            self.convex_hull
        )
        
        # Compute gradients
        vt.backward()
        
        # Extract gradients
        grads = params_tensor.grad.detach().cpu().numpy()
        
        # Logging for debugging
        logging.info(f"Gradients: {grads}")
        
        return grads  # IPOPT minimizes, so negate gradients
    # def compute_constraints(self, params_tensor):
    #     idx = 0
    #     delta_plus = params_tensor[idx : idx + self.D]
    #     delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
    #     delta = delta_plus - delta_minus

    #     # Compute constraints
    #     constraint_3 = self.xt + delta  # xt + delta >= 0
    #     total_sum = torch.sum(self.xt + delta)
    #     bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau)
    #     constraint_5 = 1.0 - (total_sum + bt)  # Total weights <= 1
    #     constraint_6 = total_sum + bt  # Total weights >= 0

    #     constraints_list = [
    #         constraint_3.view(-1),  # xt + delta constraints
    #         bt.view(-1),            # Bond holdings >= 0
    #         constraint_5.view(-1),  # Total weights <= 1
    #         constraint_6.view(-1),  # Total weights >= 0
    #     ]
    #     constraints_combined = torch.cat(constraints_list)
    #     return constraints_combined
    
    # def compute_constraints(self, params_tensor):
    #     idx = 0
    #     delta_plus = params_tensor[idx : idx + self.D]
    #     delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
    #     delta = delta_plus - delta_minus

    #     # Constraint 1 to D: xt + delta >= 0 (D scalar constraints)
    #     constraint3 = self.xt + delta  # Shape: [D]

    #     # Constraint D+1: bt >= 0 (1 scalar constraint)
    #     bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau)  # Scalar

    #     # Constraint D+2: 1 - sum(x + delta) >= 0 (1 scalar constraint)
    #     constraint5 = 1.0 - torch.sum(self.xt + delta)  # Scalar

    #     # Constraint D+3: sum(x + delta) >= 0 (1 scalar constraint)
    #     constraint6 = torch.sum(self.xt + delta)  # Scalar

    #     # Concatenate all scalar constraints into a 1D tensor
    #     constraints_combined = torch.cat([
    #         constraint3.view(-1),       # D scalar constraints
    #         bt.view(1),                 # 1 scalar constraint
    #         constraint5.view(1)        # 1 scalar constraint
    #         # constraint6.view(1)         # 1 scalar constraint
    #     ])

    #     # Logging for debugging
    #     print(f"Constraints Combined Shape: {constraints_combined.shape}")
    #     print(f"Constraints Combined: {constraints_combined}")


    #     return constraints_combined    

    def compute_constraints(self, params_tensor):
        delta_plus = params_tensor[:self.D]
        delta_minus = params_tensor[self.D:2 * self.D]
        delta = delta_plus - delta_minus

        # Constraint 1 to D: x + delta >= 0 (each component)
        constraints_x_plus_delta = self.xt + delta  # Shape: [D]

        # Constraint D+1: bt >= 0
        bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau)  # Scalar

        # Constraint D+2: 1 - sum(x + delta) >= 0
        constraint_sum_le_1 = 1.0 - torch.sum(self.xt + delta)  # Scalar

        # Constraint D+3: sum(x + delta) >= 0
        constraint_sum_ge_0 = torch.sum(self.xt + delta)  # Scalar

        # Concatenate all scalar constraints into a 1D tensor in the correct order
        constraints_combined = torch.cat([
            constraints_x_plus_delta.view(-1),    # g1, g2 for D=2
            bt.view(1),                           # g3
            constraint_sum_le_1.view(1),          # g4
            constraint_sum_ge_0.view(1)           # g5
        ])

        # Logging for debugging
        print(f"Constraints Combined Shape: {constraints_combined.shape}")
        print(f"Constraints Combined: {constraints_combined}")

        return constraints_combined
    def constraints(self, params):
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        constraints_combined = self.compute_constraints(params_tensor)
        return constraints_combined.detach().cpu().numpy()

    # def jacobian(self, params):
    #     params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
    #     constraints_combined = self.compute_constraints(params_tensor)

    #     jacobian_matrix = []

    #     # Compute gradients for each constraint
    #     for constraint in constraints_combined:
    #         grad = torch.autograd.grad(constraint, params_tensor, retain_graph=True)[0]
    #         jacobian_matrix.append(grad.detach().cpu().numpy())

    #     jacobian_array = np.vstack(jacobian_matrix)
    #     return jacobian_array.flatten()
    
    # def jacobian(self, params):
    #     # Convert parameters to a tensor with gradient tracking
    #     params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        
    #     # Compute all constraints as scalar values
    #     constraints_combined = self.compute_constraints(params_tensor)

    #     jacobian_matrix = []

    #     # Iterate over each scalar constraint to compute its gradient
    #     for constraint in constraints_combined:
    #         # Ensure the constraint is a scalar (0-dimensional tensor)
    #         constraint_scalar = constraint.squeeze()
            
    #         # Compute the gradient of the constraint w.r. to all parameters
    #         grad = torch.autograd.grad(
    #             outputs=constraint_scalar,
    #             inputs=params_tensor,
    #             retain_graph=True,
    #             create_graph=False
    #         )[0]
            
    #         # Append the gradient as a numpy array
    #         jacobian_matrix.append(grad.detach().cpu().numpy())

    #     # Stack all gradients into a 2D array (m x n) and flatten it
    #     jacobian_array = np.vstack(jacobian_matrix)
    #     return jacobian_array.flatten()    
    

    def jacobian(self, params):
        # Convert parameters to a tensor with gradient tracking
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)

        # Compute all constraints as scalar values
        constraints_combined = self.compute_constraints(params_tensor)

        # Compute the Jacobian using torch.autograd.functional.jacobian
        jacobian_tensor = torch.autograd.functional.jacobian(
            lambda x: self.compute_constraints(x),
            params_tensor
        )

        # Logging for debugging
        print(f"Jacobian Tensor Shape: {jacobian_tensor.shape}")
        print(f"Jacobian Tensor: {jacobian_tensor}")

        # Ensure the Jacobian has shape (m, n)
        if jacobian_tensor.dim() == 2 and jacobian_tensor.shape == (self.m, self.n):
            return jacobian_tensor.detach().cpu().numpy().flatten()
        else:
            raise ValueError(f"Unexpected Jacobian shape: {jacobian_tensor.shape}")
        
def solve_bellman_with_ipopt(
    D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma,
    convex_hull=None, include_consumption=False, num_starts=5, drop_tolerance=0.2
):
    best_solution = None
    best_info = None
    best_obj_val = float('-inf')
    failed_attempts = 0
    max_failed_attempts = int(num_starts-1)

    # def generate_feasible_initial_guess(X_t, D, tau, include_consumption=False):
    #     delta_plus = np.random.uniform(0, 0.1, size=D)
    #     delta_minus = np.random.uniform(0, 0.1, size=D)
    #     return np.concatenate([delta_plus, delta_minus])
    
    def generate_feasible_initial_guess(xt, D, tau, include_consumption=False):
        while True:
            # Generate random values for delta_plus and delta_minus
            delta_plus = np.random.uniform(0, 1, size=D)
            delta_minus = np.random.uniform(0, 1, size=D)

            # Ensure that delta_plus and delta_minus are within the bounds
            delta_plus = np.clip(delta_plus, 0, 1)
            delta_minus = np.clip(delta_minus, 0, 1)

            # Compute delta
            delta = delta_plus - delta_minus

            # Compute transaction costs
            transaction_costs = tau * np.sum(delta_plus - delta_minus)

            # Compute bond holdings (bt), ensuring non-negative bond holdings
            bt = 1.0 - np.sum(xt + delta) - transaction_costs
            if bt < 0:
                continue  # Retry if bond holdings are negative

            # Optionally include consumption
            c_t = 0.0 if not include_consumption else np.random.uniform(0, 0.05)

            # Verify that x + delta >= 0
            x_plus_delta = xt + delta
            if np.any(x_plus_delta < 0):
                continue  # Retry if any asset constraint is violated

            # Verify that 1 - sum(x + delta) >= 0
            if 1.0 - np.sum(x_plus_delta) < 0:
                continue  # Retry if sum constraint is violated

            # Verify no shorting constraints: delta >= -xt
            if np.any(delta < -xt):
                continue  # Retry if no shorting constraint is violated

            # Return the initial guess if all constraints are satisfied
            initial_guess = np.concatenate([delta_plus, delta_minus])
            return initial_guess

    def generate_feasible_initial_guess(X_t, D, tau, include_consumption=False):
        delta_plus = np.zeros(D)
        delta_minus = np.zeros(D)

        total_available = 1.0 - np.sum(X_t)
        if total_available < 0:
            raise ValueError("Initial x_t exceeds the total available weight (1.0).")

        for d in range(D):
            # Limit delta_plus to be within remaining available space and current asset holding
            max_delta_plus = min(X_t[d], total_available)
            delta_plus[d] = np.random.uniform(0, max_delta_plus)

            # Calculate the available room for delta_minus given delta_plus
            max_delta_minus = X_t[d] - delta_plus[d]
            delta_minus[d] = np.random.uniform(0, max_delta_minus)

            # Update the total available room for the next assets
            total_available -= delta_plus[d]

        # Compute transaction costs
        transaction_costs = tau * np.sum(delta_plus - delta_minus)

        # Compute bond holdings (bt), ensuring non-negative bond holdings
        bt = 1.0 - np.sum(X_t + delta_plus - delta_minus) - transaction_costs
        if bt < 0:
            raise ValueError("Initial guess led to infeasible bond holdings.")

        # Optionally include consumption
        c_t = 0.0 if not include_consumption else np.random.uniform(0, 0.05)

        # Verify that x + delta >= 0
        x_plus_delta = X_t + delta_plus - delta_minus
        if np.any(x_plus_delta < 0):
            raise ValueError("Initial guess does not satisfy x + delta >= 0.")

        # Verify that 1 - sum(x + delta) >=0
        if 1.0 - np.sum(x_plus_delta) < 0:
            raise ValueError("Initial guess does not satisfy sum(x + delta) <= 1.")

        # Optionally verify other constraints
        if bt < 0:
            raise ValueError("Initial guess does not satisfy bond holdings >= 0.")

        # Return the initial guess
        initial_guess = np.concatenate([delta_plus, delta_minus])
        print(f"Initial Guess: {initial_guess}")
        return initial_guess

    # Loop through multiple starting points
    for start_idx in range(num_starts):
        initial_guess = generate_feasible_initial_guess(xt.cpu().numpy(), D, tau, include_consumption)

        # delta_plus = initial_guess[:D]
        # delta_minus = initial_guess[D:2*D]
        # delta = delta_plus - delta_minus
        # x_plus_delta = xt + delta
        # # bt = normalized_bond_holdings(torch.tensor([xt]), torch.tensor([delta_plus]), torch.tensor([delta_minus]), tau)

        # print(f"x + delta: {x_plus_delta}")
        # print(f"bt: {1.0 - np.sum(x_plus_delta)}")
        # print(f"Sum(x + delta): {np.sum(x_plus_delta)}")

        try:
            # Create the optimization problem
            prob = PortfolioOptimization(
                D,
                xt,
                vt_next_in,
                vt_next_out,
                t,
                T,
                beta,
                gamma,
                tau,
                Rf,
                mu,
                Sigma,
                convex_hull=convex_hull,
                include_consumption=include_consumption,
            )

            prob.add_option("tol", 1e-6)
            prob.add_option("max_iter", 300)
            prob.add_option("print_level", 5)
            # prob.add_option("acceptable_tol", 1e-5)
            prob.add_option("honor_original_bounds", "yes")
            prob.add_option("mu_strategy", "adaptive")  # Adaptive step size strategy
            prob.add_option("mu_oracle", "quality-function")  # Control step quality
            prob.add_option("derivative_test", "first-order")
            prob.add_option("derivative_test_tol", 1e-4)            
            # Optionally disable derivative checker if it's causing issues
            # prob.add_option("derivative_test", "none")

            solution, info = prob.solve(initial_guess)

            # Check if this solution is better than the current best
            if info['status'] == 0 and (best_solution is None or info['obj_val'] > best_obj_val):
                best_solution = solution
                best_info = info
                best_obj_val = info['obj_val']

        except Exception as e:
            print(f"Optimization failed for start {start_idx}: {e}")
            failed_attempts += 1
            # If too many failures occur, drop this point
            if failed_attempts > max_failed_attempts:
                print(f"Exceeded maximum allowed failed attempts: {max_failed_attempts}")
                return None, None, None, None, None
            continue

    if best_solution is None:
        print(f"No optimizer solution found for point {xt}!")
        return None, None, None, None, None

    # After finding the best solution, extract the variables
    idx = 0
    delta_plus_opt = best_solution[idx : idx + D]
    delta_minus_opt = best_solution[idx + D : idx + 2 * D]
    delta_opt = delta_plus_opt - delta_minus_opt

    # Compute omega_i_t and bond holdings (bt)
    omega_i_t = xt.cpu().numpy() + delta_opt
    bt = normalized_bond_holdings(
        xt, torch.tensor(delta_plus_opt), torch.tensor(delta_minus_opt), tau
    ).item()

    return delta_plus_opt, delta_minus_opt, delta_opt, omega_i_t, bt

def approximate_ntr(vt_next_in, vt_next_out, D, t, T, beta, gamma, tau, Rf, mu, Sigma):
    # Step 1: Sample state points
    tilde_X_t = sample_state_points(D)
    N = len(tilde_X_t)
    tilde_omega_t = []

    for i in range(N):
        tilde_x_i_t = tilde_X_t[i]
        # Step 2: Solve optimization problem
        delta_plus, delta_minus, delta, omega_i_t, b_t = solve_bellman_with_ipopt(
            D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        )
        if delta_plus is not None:
            # Step 3: Compute NTR vertices
            tilde_omega_i_t = (tilde_x_i_t + delta).detach().cpu().numpy()
            tilde_omega_t.append(tilde_omega_i_t)

    # Step 4: Compute convex hull of the vertices to represent the NTR
    tilde_omega_t = np.array(tilde_omega_t)
    if len(tilde_omega_t) >= D + 1:
        convex_hull = ConvexHull(tilde_omega_t)
    else:
        convex_hull = None  # Cannot compute convex hull with fewer points

    return tilde_omega_t, convex_hull

def dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma):
    # Initialize value function V
    V = [[None, None] for _ in range(T + 1)]
    
    # Set terminal value function
    V[T][0] = V_terminal  # For inside NTR
    V[T][1] = V_terminal  # For outside NTR

    NTRs = [None for _ in range(T)]  # Store NTRs for each period

    for t in reversed(range(T)):
        print(f"Time step {t}")

        # Step 2a: Approximate NTR
        tilde_omega_t, convex_hull = approximate_ntr(V[t + 1][0], V[t + 1][1], D, t, T, beta, gamma, tau, Rf, mu, Sigma)
        NTRs[t] = convex_hull

        # Step 2b: Sample state points
        X_t = sample_state_points_simplex(D, N)
        data_in = []
        data_out = []

        for i in range(len(X_t)):
            x_i_t = X_t[i]
            # Step 2c: Solve optimization problem
            delta_plus, delta_minus, delta, omega_i_t, b_t = solve_bellman_with_ipopt(
                D, x_i_t, V[t + 1][0], V[t + 1][1], t, T, beta, gamma, tau, Rf, mu, Sigma,
                convex_hull=NTRs[t]
            )
            if delta_plus is None:
                continue  # Skip if optimization failed
            print(f"Time: {t}, Point: {x_i_t}, Delta+: {delta_plus}, Delta-: {delta_minus}, Delta: {delta}, Omega: {omega_i_t}, bt: {b_t}")
            # Compute value using Bellman equation
            v_i_t = bellman_equation(V[t + 1][0], V[t + 1][1], x_i_t, 
                                     torch.tensor(delta_plus), torch.tensor(delta_minus), beta, gamma, tau, Rf, convex_hull=NTRs[t])

            # Determine if the point is inside the NTR and append to the respective data set
            x_i_t_np = x_i_t.detach().cpu().numpy()
            in_ntr = is_in_ntr(x_i_t_np, convex_hull)
            if in_ntr:
                data_in.append((x_i_t_np, v_i_t.item()))
            else:
                data_out.append((x_i_t_np, v_i_t.item()))

        # Step 2e: Train GPR models for inside and outside NTR
        if data_in:
            train_x_in = torch.tensor([d[0] for d in data_in], dtype=torch.float32)
            train_y_in = torch.tensor([d[1] for d in data_in], dtype=torch.float32)
            model_in, likelihood_in = train_gp_model(train_x_in, train_y_in)
            V[t][0] = model_in
        else:
            V[t][0] = V[t + 1][0]

        if data_out:
            train_x_out = torch.tensor([d[0] for d in data_out], dtype=torch.float32)
            train_y_out = torch.tensor([d[1] for d in data_out], dtype=torch.float32)
            model_out, likelihood_out = train_gp_model(train_x_out, train_y_out)
            V[t][1] = model_out
        else:
            V[t][1] = V[t + 1][1]
    
    return V, NTRs

# Parameters
T = 6  # Time horizon
N = 100  # Number of sample points
D = 2  # Number of risky assets

# Run the dynamic programming algorithm
solve_bellman_with_ipopt(D, torch.tensor([0.49, 0.49]), V_terminal, V_terminal, 4, 5, beta, gamma, tau, Rf, mu, Sigma, convex_hull=None)
# V, NTRs = dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma)

Initial Guess: [0.00334251 0.01599493 0.0394183  0.44483018]
This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.2.

Constraints Combined Shape: torch.Size([5])
Constraints Combined: tensor([-0.3032,  0.6085,  0.6981,  0.6947,  0.3053], grad_fn=<CatBackward0>)
Constraints Combined Shape: torch.Size([5])
Constraints Combined: tensor([-0.3032,  0.6085,  0.6981,  0.6947,  0.3053], grad_fn=<CatBackward0>)
Constraints Combined Shape: torch.Size([5])
Constraints Combined: tensor([-0.3032,  0.6085,  0.6981,  0.6947,  0.3053], grad_fn=<CatBackward0>)
Jacobian Tensor Shape: torch.Size([5, 4])
Jacobian Tensor: tensor([[ 1.0000,  0.0000, -1.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000, -1.0000],
        [-1.0050, -1.0050,  1.0050,  1.0050],
        [-1.0000, -1.0000,  1.0000,  1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000]])
Starting derivative checker for first derivatives.

* grad_f[          0] =  1.1413071289062500e+03    ~  0.0000000000000000e+00  [ 1.141e+07]
Co

(array([0.12829377, 0.12985279]),
 array([0.61829377, 0.61985279]),
 array([-0.49000001, -0.49000001]),
 array([1.85865917e-09, 3.56586483e-09]),
 1.004899994643721)

In [1]:
import numpy as np
import torch
import torch.autograd as autograd

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

import cyipopt
from cyipopt import Problem
from scipy.spatial import ConvexHull

import logging

# Set up logging configuration
logging.basicConfig(filename='optimization_log.txt', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Set random seed for reproducibility
np.random.seed(2001)
torch.manual_seed(2001)

# Parameters
T = 10  # Time horizon
D = 2  # Number of risky assets
r = 0.02  # Risk-free return in pct.
# Rf = np.exp(r)  # Risk-free return
Rf = r  # Risk-free return
# Rf = np.exp(r)  # Risk-free return
tau = 0.0005  # Transaction cost rate
beta = 0.975  # Discount factor
gamma = 3.0  # Risk aversion coefficient

# Risky assets - deterministic
mu = np.array([0.07, 0.07])

Sigma = np.array([[0.2, 0], [0, 0.2]])

# Include consumption flag
include_consumption = False  # Set to True to include consumption

# Define the GPR model with ARD
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_gp_model(train_x, train_y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood(
        noise_constraint=gpytorch.constraints.GreaterThan(1e-6)
    )
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 250
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    return model, likelihood

def utility(var, gamma):
    if gamma == 1:
        return torch.log(var)  # Log utility for gamma = 1
    else:
        return (var ** (1.0 - gamma)) / (1 - gamma)  # CRRA utility

def safe_utility(var, gamma):
    var = torch.clamp(var, min=1e-10)
    return utility(var, gamma)

def normalized_bond_holdings(xt, delta_plus, delta_minus, tau):
    delta = delta_plus - delta_minus
    transaction_costs = tau * torch.sum(delta_plus - delta_minus)
    # Compute bond holdings
    bt = 1.0 - torch.sum(xt + delta) - transaction_costs
    return bt

# def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
#     delta = delta_plus - delta_minus
#     # Wealth at t+1
#     pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
#     # Portfolio weights at t+1
#     xt1 = ((xt + delta) * Rt) / pi_t1
#     xt1.requires_grad_(True)    
#     return pi_t1, xt1

def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    delta = delta_plus - delta_minus
    # Wealth at t+1
    pi_t1 = bt * Rf + torch.sum((xt + delta) * Rt)
    # Ensure pi_t1 is positive
    epsilon = 1e-8
    pi_t1 = torch.clamp(pi_t1, min=epsilon)
    # Portfolio weights at t+1
    xt1 = ((xt + delta) * Rt) / pi_t1
    # xt1.requires_grad_(True)  # Not needed here
    return pi_t1, xt1

def V_terminal(xT):
    return utility(1.0 - tau * torch.sum(torch.abs(xT)), gamma)

def bellman_equation(vt_next_in, vt_next_out, xt, delta_plus, delta_minus, beta, gamma, tau, Rf, convex_hull=None):
    # Compute bond holdings
    bt = normalized_bond_holdings(xt, delta_plus, delta_minus, tau)

    # Simulate returns (expected returns for simplicity)
    Rt = torch.tensor(mu, dtype=torch.float32)

    # Compute next period wealth dynamics
    pi_t1, xt1 = normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf)

    # Do not set requires_grad on xt1
    if  isinstance(vt_next_in, gpytorch.models.ExactGP):    
        xt1.requires_grad = True
    # Determine whether the next state is inside or outside the NTR

    if is_in_ntr(xt1.detach().cpu().numpy(), convex_hull):
        # Inside the NTR, use vt_next_in
        if isinstance(vt_next_in, gpytorch.models.ExactGP):
            vt_next_in.eval()
            # with torch.no_grad():
            vt_next_val = vt_next_in(xt1).mean.squeeze(0)
        elif callable(vt_next_in):
            vt_next_val = vt_next_in(xt1)
        elif vt_next_in is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_in to be a GP model or function.")
    else:
        # Outside the NTR, use vt_next_out
        if isinstance(vt_next_out, gpytorch.models.ExactGP):
            vt_next_out.eval()
            # with torch.no_grad():
            vt_next_val = vt_next_out(xt1).mean.squeeze(0)
        elif callable(vt_next_out):
            vt_next_val = vt_next_out(xt1)
        elif vt_next_out is None:
            vt_next_val = V_terminal(xt1)
        else:
            raise TypeError("Expected vt_next_out to be a GP model or function.")
        
    # Decide which value function to use based on NTR
    # in_ntr = is_in_ntr(xt1.detach().cpu().numpy(), convex_hull)
    # vt_val = vt_next_in(xt1).mean.squeeze(0) if in_ntr else vt_next_out(xt1).mean.squeeze(0)    

    # Compute the value function
    vt = beta * (pi_t1 ** (1.0 - gamma)) * vt_next_val
    # vt = beta * (pi_t1 ** (1.0 - gamma)) * vt_val
    # Compute the value function
    if torch.isnan(pi_t1) or torch.isinf(pi_t1):
        raise ValueError("Invalid pi_t1 encountered in bellman_equation.")
    if torch.isnan(vt_next_val) or torch.isinf(vt_next_val):
        raise ValueError("Invalid vt_next_val encountered in bellman_equation.")
    if torch.isnan(vt) or torch.isinf(vt):
        raise ValueError("Invalid vt encountered in bellman_equation.")

    vt = torch.clamp(vt, min=-1e21, max=1e21)

    return vt

def sample_state_points(D):
    from itertools import product
    # Generate all combinations of 0 and 1 for D dimensions
    points = list(product([0, 1], repeat=D))
    # Add the midpoint
    midpoint = [0.5] * D
    points.append(midpoint)
    return torch.tensor(points, dtype=torch.float32)

def sample_state_points_simplex(D, N):
    # Generate random points in the simplex
    def random_points_in_simplex(n, k):
        points = np.random.dirichlet(np.ones(k), size=n)
        return points
    points = random_points_in_simplex(N, D)
    return torch.tensor(points, dtype=torch.float32)

def is_in_ntr(x, convex_hull):
    if convex_hull is None:
        return False
    new_point = np.array(x)
    hull = convex_hull
    A = hull.equations[:, :-1]
    b = -hull.equations[:, -1]
    inequalities = np.dot(A, new_point) + b
    return np.all(inequalities <= 1e-5)  # Allow for numerical tolerance

def MertonPoint(mu, Sigma, r, gamma):
    # Compute the Merton portfolio weights
    Lambda = np.diag(np.sqrt(np.diag(Sigma)))
    Lambda_Sigma_Lambda = np.dot(Lambda, np.dot(Sigma, Lambda))
    Lambda_Sigma_Lambda_inv = np.linalg.inv(Lambda_Sigma_Lambda)
    mu_r = mu - r
    pi = np.dot(Lambda_Sigma_Lambda_inv, mu_r / gamma)
    return pi

# Define the PortfolioOptimization class without inheriting from cyipopt.Problem
class PortfolioOptimization:
    def __init__(
        self,
        D,
        xt,
        vt_next_in,
        vt_next_out,
        t,
        T,
        beta,
        gamma,
        tau,
        Rf,
        mu,
        Sigma,
        convex_hull=None,
        include_consumption=False,
        ntr_mid_point=None
    ):
        self.D = D
        self.xt = xt.detach().clone()  # Ensure self.xt is a leaf variable
        self.vt_next_in = vt_next_in
        self.vt_next_out = vt_next_out
        self.t = t
        self.T = T
        self.beta = beta
        self.gamma = gamma
        self.tau = tau
        self.Rf = Rf
        self.mu = mu
        self.Sigma = Sigma
        self.convex_hull = convex_hull
        self.include_consumption = include_consumption
        self.ntr_mid_point = ntr_mid_point

        # Number of variables: delta_plus, delta_minus
        self.n = 2*D

        # Number of constraints: D constraints from xt + delta >= 0, and 3 scalar constraints
        # self.m = 2 * D + 3
        self.m = 2*D + 3


    def objective(self, params):
        idx = 0
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)

        delta_plus = params_tensor[idx : idx + self.D]
        delta_minus = params_tensor[idx + self.D : idx + 2 * self.D]
        # Compute the value function
        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            delta_plus,
            delta_minus,
            self.beta,
            self.gamma,
            self.tau,
            self.Rf,
            self.convex_hull
        )
        
        if torch.isnan(vt).any() or torch.isinf(vt).any():
            raise ValueError("NaN or Inf detected in objective function!")
        
        # Logging for debugging
        logging.info(f"delta_plus: {delta_plus.detach().cpu().numpy()}, delta_minus: {delta_minus.detach().cpu().numpy()}")
        logging.info(f"Objective Value (vt): {vt.item()}")
        
        return -vt.item()  # IPOPT minimizes, so negate the value
        
        
    def gradient(self, params):
        # Convert params to a tensor with gradient tracking
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        # delta_plus = params_tensor[:self.D]
        # delta_minus = params_tensor[self.D:2 * self.D]
        
        # Compute the value function
        vt = bellman_equation(
            self.vt_next_in,
            self.vt_next_out,
            self.xt,
            params_tensor[:self.D],
            params_tensor[self.D:2 * self.D],
            self.beta,
            self.gamma,
            self.tau,
            self.Rf,
            self.convex_hull
        )
        
        # Compute gradients
        vt.backward()
        
        # Extract gradients
        grads = params_tensor.grad.detach().cpu().numpy()
        
        # Logging for debugging
        logging.info(f"Gradients: {grads}")
        
        return -grads  # IPOPT minimizes, so negate gradients

    def validate_gradients(portfolio_opt, params):
        eps = 1e-6
        analytical_grads = portfolio_opt.gradient(params)
        numerical_grads = np.zeros_like(analytical_grads)
        
        for i in range(len(params)):
            params_eps_plus = params.copy()
            params_eps_plus[i] += eps
            obj_plus = portfolio_opt.objective(params_eps_plus)
            
            params_eps_minus = params.copy()
            params_eps_minus[i] -= eps
            obj_minus = portfolio_opt.objective(params_eps_minus)
            
            numerical_grads[i] = (obj_plus - obj_minus) / (2 * eps)
        
        # Compare gradients
        if not np.allclose(analytical_grads, numerical_grads, atol=1e-4):
            logging.error("Analytical and numerical gradients do not match!")
        else:
            logging.info("Gradients are correctly computed.")

    def constraints(self, params):
        # Convert params to tensors without gradient tracking
        delta_plus = torch.tensor(params[:self.D], dtype=torch.float32)
        delta_minus = torch.tensor(params[self.D:2 * self.D], dtype=torch.float32)
        delta = delta_plus - delta_minus

        # Compute transaction costs
        transaction_costs = self.tau * torch.sum(delta_plus - delta_minus)

        # Compute constraints
        constraints_x_plus_delta = self.xt + delta  # Shape: [D]

        # Bond Holdings Constraint: b_t >=0
        bt = 1.0 - torch.sum(self.xt + delta) - transaction_costs

        # Portfolio Sum Constraint: sum(x + delta) + tau * sum(delta_plus - delta_minus) <=1
        constraint_sum_le_1 = 1.0 - torch.sum(self.xt + delta) - transaction_costs

        # Sum(x + delta) >=0
        constraint_sum_ge_0 = torch.sum(self.xt + delta)  # sum(x + delta) >=0

        # No Shorting Constraints: delta >= -x_t
        constraints_no_shorting = delta + self.xt  # Shape: [D]

        # Concatenate all constraints
        constraints_combined = torch.cat([
            constraints_x_plus_delta.view(-1),    # D constraints: x_t + delta >=0
            torch.tensor([bt], dtype=torch.float32),  # 1 constraint: b_t >=0
            torch.tensor([constraint_sum_le_1], dtype=torch.float32),  # 1 constraint: sum(x + delta) + tau * sum(delta_plus - delta_minus) <=1
            torch.tensor([constraint_sum_ge_0], dtype=torch.float32),  # 1 constraint: sum(x + delta) >=0
            constraints_no_shorting.view(-1)  # D constraints: delta >= -x_t
        ])

        # Logging for debugging
        logging.info(f"Constraints Combined Shape: {constraints_combined.shape}")
        logging.info(f"Constraints Combined: {constraints_combined.detach().cpu().numpy()}")

        return constraints_combined.detach().cpu().numpy()
    # def jacobian(self, params):
    #     # Convert parameters to a tensor with gradient tracking
    #     params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)

    #     # Define a function to compute constraints
    #     def compute_constraints_func(x):
    #         delta_plus = x[:self.D]
    #         delta_minus = x[self.D:2 * self.D]
    #         delta = delta_plus - delta_minus

    #         constraints_x_plus_delta = self.xt + delta
    #         bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau)
    #         constraint_sum_le_1 = 1.0 - torch.sum(self.xt + delta)
    #         constraint_sum_ge_0 = torch.sum(self.xt + delta)

    #         constraints_combined = torch.cat([
    #             constraints_x_plus_delta.view(-1),
    #             bt.view(1),
    #             constraint_sum_le_1.view(1),
    #             constraint_sum_ge_0.view(1)
    #         ])

    #         return constraints_combined

    #     # Compute the Jacobian using torch.autograd.functional.jacobian
    #     jacobian_tensor = torch.autograd.functional.jacobian(compute_constraints_func, params_tensor)

    #     # Logging for debugging
    #     logging.info(f"Jacobian Tensor Shape: {jacobian_tensor.shape}")
    #     logging.info(f"Jacobian Tensor: {jacobian_tensor}")

    #     # Ensure the Jacobian has shape (m, n)
    #     if jacobian_tensor.dim() == 2 and jacobian_tensor.shape == (self.m, self.n):
    #         # Flatten in column-major order
    #         return jacobian_tensor.detach().cpu().numpy().flatten(order='F')
    #     else:
    #         raise ValueError(f"Unexpected Jacobian shape: {jacobian_tensor.shape}")
        
    def jacobian(self, params):
        # Convert params to tensor with gradient tracking
        params_tensor = torch.tensor(params, dtype=torch.float32, requires_grad=True)
        delta_plus = params_tensor[:self.D]
        delta_minus = params_tensor[self.D:2 * self.D]
        delta = delta_plus - delta_minus

        # Compute constraints
        constraints_x_plus_delta = self.xt + delta  # Shape: [D]
        bt = normalized_bond_holdings(self.xt, delta_plus, delta_minus, self.tau)  # Scalar
        sum_x_plus_delta = torch.sum(self.xt + delta)  # Scalar
        constraint_sum_le_1 = 1.0 - sum_x_plus_delta  # sum(x + delta) <=1
        constraint_sum_ge_0 = sum_x_plus_delta  # sum(x + delta) >=0
        constraints_no_shorting = delta + self.xt  # Shape: [D]

        # Concatenate all constraints
        constraints_combined = torch.cat([
            constraints_x_plus_delta.view(-1),
            torch.tensor([bt], dtype=torch.float32),
            torch.tensor([constraint_sum_le_1], dtype=torch.float32),
            torch.tensor([constraint_sum_ge_0], dtype=torch.float32),
            constraints_no_shorting.view(-1)
        ])

        # Compute Jacobian using autograd
        jacobian_rows = []
        for constraint in constraints_combined:
            grads = autograd.grad(constraint, params_tensor, retain_graph=True, create_graph=False)
            jacobian_rows.append(grads[0].detach().cpu().numpy())

        jacobian_matrix = np.array(jacobian_rows)
        # Flatten in column-major order as IPOPT expects column-wise flattening
        return jacobian_matrix.flatten(order='F')
    
def solve_bellman_with_ipopt(
    D, xt, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma,
    convex_hull=None, ntr_mid_point=None, include_consumption=False, num_starts=10, drop_tolerance=0.2
):
    best_solution = None
    best_info = None
    best_obj_val = float('-inf')
    failed_attempts = 0
    max_failed_attempts = int(num_starts)

    # def generate_feasible_initial_guess(X_t, D, tau, include_consumption=False):
    #     delta_plus = np.zeros(D)
    #     delta_minus = np.zeros(D)

    #     # Compute the maximum allowable sum(x + delta) + tau * sum(delta_plus - delta_minus) <=1
    #     max_sum = 1.0 - tau * np.sum(X_t)  # Simplistic approach; refine as needed

    #     if max_sum < 0:
    #         raise ValueError("Initial x_t and transaction costs exceed the total allowable weight (1.0).")

    #     for d in range(D):
    #         # Allocate delta_plus and delta_minus proportionally or using a heuristic
    #         delta_plus[d] = np.random.uniform(0, X_t[d])
    #         delta_minus[d] = np.random.uniform(0, X_t[d])

    #     # Compute transaction costs
    #     transaction_costs = tau * np.sum(delta_plus - delta_minus)

    #     # Compute bond holdings (bt), ensuring non-negative bond holdings
    #     bt = 1.0 - np.sum(X_t + delta_plus - delta_minus) - transaction_costs
    #     if bt < 0:
    #         # Scale down deltas proportionally to make bt >=0
    #         scaling_factor = (1.0 - np.sum(X_t) - transaction_costs) / np.sum(delta_plus - delta_minus)
    #         delta_plus *= scaling_factor
    #         delta_minus *= scaling_factor
    #         bt = 1.0 - np.sum(X_t + delta_plus - delta_minus) - transaction_costs
    #         if bt < 0:
    #             raise ValueError("Initial guess led to infeasible bond holdings.")

    #     # Optionally include consumption
    #     c_t = 0.0 if not include_consumption else np.random.uniform(0, 0.05)

    #     # Verify that x + delta >= 0
    #     x_plus_delta = X_t + delta_plus - delta_minus
    #     if np.any(x_plus_delta < 0):
    #         raise ValueError("Initial guess does not satisfy x + delta >= 0.")

    #     # Verify that 1 - sum(x + delta) >=0
    #     if 1.0 - np.sum(x_plus_delta) < 0:
    #         raise ValueError("Initial guess does not satisfy sum(x + delta) <= 1.")

    #     # Optionally verify other constraints
    #     if bt < 0:
    #         raise ValueError("Initial guess does not satisfy bond holdings >= 0.")

    #     # Return the initial guess
    #     initial_guess = np.concatenate([delta_plus, delta_minus])
    #     logging.info(f"Initial Guess: {initial_guess}")
    #     return initial_guess    
    def generate_feasible_initial_guess(X_t, D, tau, include_consumption=False):
        max_attempts = 100
        attempt = 0
        while attempt < max_attempts:
            try:
                delta_plus = np.random.uniform(0, X_t, size=D)
                delta_minus = np.random.uniform(0, X_t, size=D)

                # Compute transaction costs
                transaction_costs = tau * np.sum(delta_plus - delta_minus)

                # Compute bond holdings (bt), ensuring non-negative bond holdings
                bt = 1.0 - np.sum(X_t + delta_plus - delta_minus) - transaction_costs

                # Check feasibility
                if bt >= 0 and (X_t + delta_plus - delta_minus >= 0).all() and (np.sum(X_t + delta_plus - delta_minus) + tau * np.sum(delta_plus - delta_minus) <= 1):
                    initial_guess = np.concatenate([delta_plus, delta_minus])
                    logging.info(f"Initial Guess: {initial_guess}")
                    return initial_guess

                # If not feasible, adjust deltas incrementally
                scale_factor = 0.9 ** attempt  # Decrease the scale exponentially
                delta_plus *= scale_factor
                delta_minus *= scale_factor
                attempt += 1

            except Exception as e:
                logging.warning(f"Attempt {attempt}: {e}")
                attempt += 1

        raise ValueError("Failed to generate a feasible initial guess after multiple attempts.")
    print(f"point of optimization: {xt}")
    if torch.sum(xt) == 0:
        xt = torch.tensor([1e-8, 1e-8])
    # Loop through multiple starting points
    for start_idx in range(num_starts):
        try:
            initial_guess = generate_feasible_initial_guess(xt.cpu().numpy(), D, tau, include_consumption=False)
        except ValueError as e:
            logging.warning(f"Start {start_idx}: {e}")
            failed_attempts += 1
            if failed_attempts > max_failed_attempts:
                logging.error(f"Exceeded maximum allowed failed attempts: {max_failed_attempts}")
                return None, None, None, None, None
            continue

        try:
            # Create an instance of PortfolioOptimization
            portfolio_opt = PortfolioOptimization(
                D=D,
                xt=xt,
                vt_next_in=vt_next_in,
                vt_next_out=vt_next_out,
                t=t,
                T=T,
                beta=beta,
                gamma=gamma,
                tau=tau,
                Rf=Rf,
                mu=mu,
                Sigma=Sigma,
                convex_hull=convex_hull,
                ntr_mid_point=ntr_mid_point,  # Pass ntr_mid_point
                include_consumption=include_consumption,
            )

            # Define variable bounds
            lb = np.zeros(2 * D)
            ub = np.ones(2 * D)            

            xt_np = xt.detach().cpu().numpy()

            # if ntr_mid_point is not None:
            #     for i in range(D):
            #         if xt_np[i] < ntr_mid_point[i]:
            #             # Allow buying only, delta_minus_i = 0
            #             lb[i] = 0       # delta_plus_i >= 0
            #             ub[i] = 1       # delta_plus_i <= 1
            #             lb[D + i] = 0
            #             ub[D + i] = 0   # delta_minus_i = 0
            #         elif xt_np[i] > ntr_mid_point[i]:
            #             # Allow selling only, delta_plus_i = 0
            #             lb[i] = 0
            #             ub[i] = 0       # delta_plus_i = 0
            #             lb[D + i] = 0
            #             ub[D + i] = 1   # delta_minus_i >= 0
            #         else:
            #             # Neither buying nor selling
            #             lb[i] = 0
            #             ub[i] = 0       # delta_plus_i = 0
            #             lb[D + i] = 0
            #             ub[D + i] = 0   # delta_minus_i = 0
            # else:
            #     # If ntr_mid_point is not available, allow both buying and selling (initial periods)
            #     lb = np.zeros(2 * D)
            #     ub = np.ones(2 * D)
            lb = np.zeros(2 * D)
            ub = np.ones(2 * D)

            # Define constraint bounds
            cl = np.zeros(portfolio_opt.m)
            cu = np.full(portfolio_opt.m, np.inf)

            # Bond Holdings Constraint: b_t >=0
            cl[2] = 0.0  # Constraint index 2 corresponds to b_t >=0
            cu[2] = np.inf

            # Portfolio Sum Constraint: sum(x + delta) + tau * sum(delta_plus - delta_minus) <=1
            cl[3] = -np.inf  # No lower bound
            cu[3] = 1.0      # Upper bound

            # Sum(x + delta) >=0
            cl[4] = 0.0      # Constraint index 4 corresponds to sum(x + delta) >=0
            cu[4] = np.inf

            # No Shorting Constraints: delta >= -x_t for each asset
            for i in range(D):
                cl[5 + i] = -xt[i].item()  # delta_i >= -x_t_i


            # cu[D:2*D] = xt_np + 1e-6 # Upper bounds for delta_minus_le_xt constraints            
            # ub[D:2*D] = xt_np
            # # For delta_minus <= x_t constraints, set upper bounds accordingly
            # for i in range(D, 2 * D):
            #     cu[i] = xt_np[i - D]  # Upper bound for delta_minus_i <= x_t_i


            # Instantiate IPOPT Problem
            prob = Problem(
                n=portfolio_opt.n,
                m=portfolio_opt.m,
                problem_obj=portfolio_opt,
                lb=lb,
                ub=ub,
                cl=cl,
                cu=cu
            )

            # Set IPOPT options
            # prob.add_option("tol", 1e-6)
            # prob.add_option("max_iter", 1000)
            # prob.add_option("print_level", 3)
            # prob.add_option("acceptable_tol", 1e-5)
            # prob.add_option("honor_original_bounds", "yes")
            # prob.add_option("mu_strategy", "adaptive")  # Adaptive step size strategy
            # prob.add_option("mu_oracle", "quality-function")  # Control step quality
            # prob.add_option("derivative_test", "none")  # Temporarily disable to see if it affects feasibility            
            # prob.add_option("derivative_test", "first-order")
            # prob.add_option("derivative_test_tol", 1e-4)
            # prob.add_option("nlp_scaling_method", "gradient-based")  # Scaling method
            # prob.add_option("accept_every_trial_step", "no")
            # Solve the optimization problem
            solution, info = prob.solve(initial_guess)

            # Check if this solution is better than the current best
            if info['status'] == 0 and (best_solution is None or info['obj_val'] > best_obj_val):
                best_solution = solution
                best_info = info
                best_obj_val = info['obj_val']

        except Exception as e:
            print(f"Optimization failed for start {start_idx}: {e}")
            failed_attempts += 1
            if failed_attempts > max_failed_attempts:
                logging.error(f"Exceeded maximum allowed failed attempts: {max_failed_attempts}")
                return None, None, None, None, None
            continue

    print(f"delta_plus: {best_solution[:D]}, delta_minus: {best_solution[D:2 * D]}, omega: {xt.cpu().numpy() + best_solution[:D] - best_solution[D:2 * D]}")

    if best_solution is None:
        print(f"No optimizer solution found for point {xt}!")
        return None, None, None, None, None

    # After finding the best solution, extract the variables
    delta_plus_opt = best_solution[:D]
    delta_minus_opt = best_solution[D:2 * D]
    delta_opt = delta_plus_opt - delta_minus_opt

    # Compute omega_i_t and bond holdings (bt)
    omega_i_t = xt.cpu().numpy() + delta_opt
    bt = normalized_bond_holdings(
        xt, torch.tensor(delta_plus_opt, dtype=torch.float32), torch.tensor(delta_minus_opt, dtype=torch.float32), tau
    ).item()

    return delta_plus_opt, delta_minus_opt, delta_opt, omega_i_t, bt

def approximate_ntr(vt_next_in, vt_next_out, D, t, T, beta, gamma, tau, Rf, mu, Sigma):
    # Step 1: Sample state points
    tilde_X_t = sample_state_points(D)
    N = len(tilde_X_t)
    tilde_omega_t = []

    for i in range(N):
        tilde_x_i_t = tilde_X_t[i]
        # Step 2: Solve optimization problem
        delta_plus, delta_minus, delta, omega_i_t, b_t = solve_bellman_with_ipopt(
            D, tilde_x_i_t, vt_next_in, vt_next_out, t, T, beta, gamma, tau, Rf, mu, Sigma
        )
        if delta_plus is not None:
            # Step 3: Compute NTR vertices
            tilde_omega_i_t = (tilde_x_i_t + delta).detach().cpu().numpy()
            tilde_omega_t.append(tilde_omega_i_t)
    print(f"Time: {t}, Point: {tilde_x_i_t}, Delta+: {delta_plus}, Delta-: {delta_minus}, Delta: {delta}, Omega: {tilde_omega_t}")

    # Step 4: Compute convex hull of the vertices to represent the NTR
    tilde_omega_t = np.array(tilde_omega_t)
    if len(tilde_omega_t) >= D + 1:
        convex_hull = ConvexHull(tilde_omega_t)
        # Compute NTR mid-point
        ntr_mid_point = np.mean(tilde_omega_t, axis=0)
    else:
        convex_hull = None  # Cannot compute convex hull with fewer points
        ntr_mid_point = None
    
    return tilde_omega_t, convex_hull, ntr_mid_point

def dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma):
    # Initialize value function V
    V = [[None, None] for _ in range(T + 1)]
    
    # Set terminal value function
    V[T][0] = V_terminal  # For inside NTR
    V[T][1] = V_terminal  # For outside NTR

    NTRs = [None for _ in range(T)]  # Store NTRs for each period

    for t in reversed(range(T)):
        print(f"Time step {t}")

        # Step 2a: Approximate NTR
        tilde_omega_t, convex_hull, ntr_mid_point = approximate_ntr(V[t + 1][0], V[t + 1][1], D, t, T, beta, gamma, tau, Rf, mu, Sigma)
        NTRs[t] = convex_hull

        # Step 2b: Sample state points
        X_t = sample_state_points_simplex(D, N)
        data_in = []
        data_out = []

        for i in range(len(X_t)):
            x_i_t = X_t[i]
            # Step 2c: Solve optimization problem
            delta_plus, delta_minus, delta, omega_i_t, b_t = solve_bellman_with_ipopt(
                D, x_i_t, V[t + 1][0], V[t + 1][1], t, T, beta, gamma, tau, Rf, mu, Sigma,
                convex_hull=NTRs[t], ntr_mid_point=ntr_mid_point
            )
            if delta_plus is None:
                continue  # Skip if optimization failed
            print(f"Time: {t}, Point: {x_i_t}, Delta+: {delta_plus}, Delta-: {delta_minus}, Delta: {delta}, Omega: {omega_i_t}, bt: {b_t}")
            # Compute value using Bellman equation
            v_i_t = bellman_equation(V[t + 1][0], V[t + 1][1], x_i_t, 
                                     torch.tensor(delta_plus), torch.tensor(delta_minus), beta, gamma, tau, Rf, convex_hull=NTRs[t])

            # Determine if the point is inside the NTR and append to the respective data set
            x_i_t_np = x_i_t.detach().cpu().numpy()
            in_ntr = is_in_ntr(x_i_t_np, convex_hull)
            if in_ntr:
                data_in.append((x_i_t_np, v_i_t.item()))
            else:
                data_out.append((x_i_t_np, v_i_t.item()))

        # Step 2e: Train GPR models for inside and outside NTR
        if data_in:
            train_x_in = torch.tensor([d[0] for d in data_in], dtype=torch.float32)
            train_y_in = torch.tensor([d[1] for d in data_in], dtype=torch.float32)
            model_in, likelihood_in = train_gp_model(train_x_in, train_y_in)
            V[t][0] = model_in
        else:
            V[t][0] = V[t + 1][0]

        if data_out:
            train_x_out = torch.tensor([d[0] for d in data_out], dtype=torch.float32)
            train_y_out = torch.tensor([d[1] for d in data_out], dtype=torch.float32)
            model_out, likelihood_out = train_gp_model(train_x_out, train_y_out)
            V[t][1] = model_out
        else:
            V[t][1] = V[t + 1][1]
    
    return V, NTRs

# Parameters
T = 6  # Time horizon
N = 100  # Number of sample points
D = 2  # Number of risky assets

# Run the dynamic programming algorithm
# V, NTRs = dynamic_programming(T, N, D, gamma, beta, tau, Rf, mu, Sigma)
solve_bellman_with_ipopt(D, torch.tensor([0.49, 0.49]), V_terminal, V_terminal, 4, 5, beta, gamma, tau, Rf, mu, Sigma, convex_hull=None)


point of optimization: tensor([0.4900, 0.4900])

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       28
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints........

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot_ntr_at_time(NTR_history, t):
    hull = NTR_history[t]
    
    if hull is not None:
        vertices = hull.points  # Vertices are stored in the 'points' attribute of ConvexHull
        D = vertices.shape[1]  # Dimension of the state space
        plt.figure()

        if D == 2:
            # 2D plot
            for simplex in hull.simplices:
                plt.plot(vertices[simplex, 0], vertices[simplex, 1], 'k-')
            plt.fill(vertices[hull.vertices, 0], vertices[hull.vertices, 1], 'lightgray', alpha=0.5)
            plt.scatter(vertices[:, 0], vertices[:, 1], color='red')  # Plot the vertices
            plt.title(f'NTR at time {t}')
            plt.xlabel('State dimension 1')
            plt.ylabel('State dimension 2')
            plt.xlim(0, 1)
            plt.ylim(0, 1)
        
        elif D == 3:
            # 3D plot
            ax = plt.axes(projection='3d')
            ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], color='red')
            ax.add_collection3d(Poly3DCollection(vertices[hull.simplices], facecolors='lightgray', edgecolors='k', alpha=0.4))
            ax.set_xlabel('State dimension 1')
            ax.set_ylabel('State dimension 2')
            ax.set_zlabel('State dimension 3')
            plt.title(f'NTR at time {t}')
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.set_zlim(0, 1)
        
        plt.show()

    else:
        print(f"Not enough vertices to form an NTR at time {t}")

# Example: Plot NTR at time t=1
plot_ntr_at_time(NTRs, 2)

In [ ]:
# sample_state_points(2)
# def sample_ntr_vertices(D):
#     from itertools import product
#     # Generate all combinations of 0 and 1 for D dimensions
#     points = list(product([0, 1], repeat=D))
#     # Add the midpoint
#     midpoint = [0.5] * D
#     points.append(midpoint)
#     return torch.tensor(points, dtype=torch.float32)
# sample_ntr_vertices(2)

In [ ]:
# EXIT: Stopping optimization at current point as requested by user.
# gradient: [-inf  inf  inf -inf -inf]
# params: [0.         0.11060591 0.09820718 0.03102945 0.10420767]
# vt: -6.414915919780255e+32
# c_t: 0.0
# delta_plus: tensor([0.1106, 0.0982], requires_grad=True)
# delta_minus: tensor([0.0310, 0.1042], requires_grad=True)
# x_t: tensor([0.0711, 0.9289])
# vt_next_in: None
# vt_next_out: <function V_terminal at 0x303b28040>


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot_ntr_at_time(NTR_history, t):
    hull = NTR_history[t]
    
    if hull is not None:
        vertices = hull.points  # Vertices are stored in the 'points' attribute of ConvexHull
        D = vertices.shape[1]  # Dimension of the state space
        plt.figure()

        if D == 2:
            # 2D plot
            for simplex in hull.simplices:
                plt.plot(vertices[simplex, 0], vertices[simplex, 1], 'k-')
            plt.fill(vertices[hull.vertices, 0], vertices[hull.vertices, 1], 'lightgray', alpha=0.5)
            plt.scatter(vertices[:, 0], vertices[:, 1], color='red')  # Plot the vertices
            plt.title(f'NTR at time {t}')
            plt.xlabel('State dimension 1')
            plt.ylabel('State dimension 2')
            plt.xlim(0, 0.75)
            plt.ylim(0, 0.75)
        
        elif D == 3:
            # 3D plot
            ax = plt.axes(projection='3d')
            ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], color='red')
            ax.add_collection3d(Poly3DCollection(vertices[hull.simplices], facecolors='lightgray', edgecolors='k', alpha=0.4))
            ax.set_xlabel('State dimension 1')
            ax.set_ylabel('State dimension 2')
            ax.set_zlabel('State dimension 3')
            plt.title(f'NTR at time {t}')
            ax.set_xlim(0, 0.6)
            ax.set_ylim(0, 0.6)
            ax.set_zlim(0, 0.6)
        
        plt.show()

    else:
        print(f"Not enough vertices to form an NTR at time {t}")

# Example: Plot NTR at time t=5
plot_ntr_at_time(NTRs,1)

In [ ]:
NTRs